In [58]:
# =============================================================================
# BLOCO 1: CONFIGURAÇÃO E CARREGAMENTO DOS DADOS (COM QUERIES)
# =============================================================================
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os
import pyodbc
import pandas as pd


print("=" * 80)
print("GESTOR DE CAMPANHAS E GESTOR DE ESCASSEZ")
print("=" * 80)

print("\n1. CONFIGURAÇÃO INICIAL E CONSULTAS AOS DADOS")

# Configurações gerais
print("\n1. CONFIGURAÇÃO")

PATH_PLANO = r'C:\Users\COTAGO\Desktop\Base\plano.csv'
FOLDER_OUTPUT = r'C:\Users\COTAGO\Desktop\Base\Export_Campanhas'
SEPARADOR = ';'
ENCODING = 'utf-8'
CAMPANHA_PRIORITARIA = 'iXS_PCDPMTOP'  # Campanha prioritária

RANDOM_SEED = 42  # Random seed para garantir reprodutibilidade

# Criar a pasta de saída, se necessário
os.makedirs(FOLDER_OUTPUT, exist_ok=True)

# Configurar datas
hoje = datetime.now()
hoje_str = hoje.strftime('%Y-%m-%d')
hoje_file = hoje.strftime('%Y%m%d_%H%M%S')
mes_atual = int(hoje.strftime('%Y%m'))  # Ano e mês atuais em formato YYYYMM
mes_anterior_num = hoje.month - 1
ano_anterior = hoje.year
if mes_anterior_num <= 0:
    mes_anterior_num = 12
    ano_anterior -= 1
mes_anterior = int(f"{ano_anterior}{mes_anterior_num:02d}")

print(f"✓ Data Atual: {hoje_str}")
print(f"✓ Mês Atual: {mes_atual}")
print(f"✓ Mês Anterior: {mes_anterior}")
print(f"✓ Campanha Prioritária: {CAMPANHA_PRIORITARIA}")

# =============================================================================
# 1.1 Queries para Carregar os Dados
# =============================================================================

# Aqui, você deve substituir pelos scripts SQL que retornam os DataFrames desejados.
# Por exemplo: df_baseenvio = pd.read_sql(query_base_envio, conexao_sql)

# ------------------------ Consulta da BaseEnvio ------------------------------
# Substitua pela QUERY específica para obter a base de envio.
# Exemplo de query:

conn_str = (
    'Driver={SQL Server};'
    'Server=Diomedes;'
    'Database=tempdb;'
    'trusted_connection=yes;' 
)

conn = pyodbc.connect(conn_str)
cursor = conn.cursor()

query_base_envio = """
/* =========================================================
   QUERY SIMPLIFICADA - CAMPANHAS DE ELEGIBILIDADE
   Lógica reorganizada para melhor clareza e manutenção
========================================================= */

-- 1. TABELAS TEMPORÁRIAS PARA GESTÃO DE DEPENDÊNCIAS
DROP TABLE IF EXISTS #temp_tables;
DECLARE @temp_tables TABLE (table_name NVARCHAR(100));
INSERT INTO @temp_tables VALUES
('#tab_max_dt_eleg'),('#SemSolvabilidade'),('#proposta_comercial_CCR'),('#UniversoAux'),('#CRC5G'),
('#Consolida'),('#Reembolsos'),('#UniversoCCR'),('#Obra01'),('#TabelaFinal'),('#TabelaFinalCCR'),
('#Cruzamento'),('#Universo'),('#BaseConsolidada'),('#BaseFinal'),('#Resultado_Financeiro'),
('#BaseEnvio'),('#PMS'),('#proposta_comercial_RUC'),('#proposta_comercial_PCD_Intermedios'),
('#proposta_comercial_PCD'),('#proposta_comercial_PCDAuto'),('#proposta_comercial_PMS'),('#ElegiveisPMS');

DECLARE @table_name NVARCHAR(100);
DECLARE drop_cursor CURSOR FOR SELECT table_name FROM @temp_tables;
OPEN drop_cursor;
FETCH NEXT FROM drop_cursor INTO @table_name;
WHILE @@FETCH_STATUS = 0
BEGIN
    EXEC('DROP TABLE IF EXISTS ' + @table_name);
    FETCH NEXT FROM drop_cursor INTO @table_name;
END
CLOSE drop_cursor;
DEALLOCATE drop_cursor;

/* =========================================================
   1. DATA MÁXIMA DE ELEGIBILIDADE
========================================================= */
SELECT MAX(IdDia) AS max_dt
INTO #tab_max_dt_eleg
FROM armada.dbo.DM_CampanhasElegibilidade;

/* =========================================================
   2. CLIENTES SEM DADOS DE SOLVABILIDADE
========================================================= */
SELECT DISTINCT 
    ce.NIF,
    CASE 
        WHEN ce.Rendimento IS NULL OR ce.DespesasSolv IS NULL THEN 100 
        ELSE ce.MensMaxSolvabilidade 
    END AS MensMaxSolvabilidadeNEW,
    0 AS excl_solvNEW
INTO #SemSolvabilidade
FROM armada.dbo.DM_CampanhasElegibilidade ce
INNER JOIN #tab_max_dt_eleg e ON ce.IdDia = e.max_dt
WHERE ce.Rendimento IS NULL 
   OR ce.DespesasSolv IS NULL 
   OR ce.TxSolvabilidade IS NULL;

/* =========================================================
   3. APURAMENTO CCR - FUNÇÕES MODULARIZADAS
========================================================= */

-- 3.1. TARIFICAÇÃO
SELECT
    a.AMOUNT AS Montante,
    a.tan,
    a.TAEG,
    a.DEADLINE AS Prazo,
    a.MONTHLYPAYWITHINSURANCE AS MM,
    a.MONTHLYPAYWITHINSURANCE - a.MONTHLYPAYWITHOUTINSURANCE AS Seguro,
    a.MTIC
INTO #proposta_comercial_CCR
FROM armada.dbo.ST_DM_CondicoesSimulacao a
JOIN armada.dbo.DM_Oferta b ON a.idoffer = b.idOferta
JOIN armada.dbo.IDH_Dim_Produto c ON b.Produto = c.CodProdutoDAH
WHERE c.CodProduto = 'CCD'
  AND a.AMOUNT IN (5000,6000,7000,8000,9000,10000,11000,12000,13000,14000,
                   15000,16000,17000,18000,19000,20000,21000,22000,23000,24000,
                   25000,26000,27000,28000,29000,30000,35000,40000,45000,50000)
  AND a.DEADLINEWITHINSURANCE = 84;

CREATE CLUSTERED INDEX IX_Tarificacao ON #proposta_comercial_CCR (Montante);

-- 3.2. UNIVERSO BASE FILTRADO
SELECT
    a1.NIF,
    CASE 
        WHEN a1.AtividadeCredito = 'Terminado' THEN 'Terminados' 
        ELSE 'SegEmCurso' 
    END AS Segmento,
    COALESCE(a1.DividaRevolving,0) + COALESCE(a1.DividaCartao,0) + 
    COALESCE(a1.DividaClassico,0) + COALESCE(a1.DividaAuto,0) AS DividaCRC,
    a1.PrestMes,
    a1.MesesTerminoMinimo
INTO #UniversoAux
FROM armada.dbo.DM_CampanhasElegibilidade a1
INNER JOIN #tab_max_dt_eleg e ON a1.IdDia = e.max_dt
WHERE a1.excl_Financiavel = 0 
  AND a1.excl_ENI = 0 AND a1.excl_SocioGerente = 0 AND a1.excl_Nacionalidade = 0
  AND a1.excl_PNM = 0 AND a1.excl_Impagos = 0 AND a1.MesesUltAbertura >= 4
  AND a1.excl_TipoResidencia = 0 AND a1.excl_rend = 0 AND a1.excl_DO = 0
  AND a1.excl_DSTI = 0 AND a1.excl_solv = 0 AND a1.excl_idade = 0
  AND a1.excl_RevolvingAtivo = 0 AND a1.excl_UltAbert_Conso = 0
  AND a1.excl_PRD = 0 AND a1.excl_removidos = 0
  AND a1.excl_RequerInvestigacao = 0 AND a1.excl_prof_instavel = 0;

CREATE CLUSTERED INDEX IX_UniversoAux ON #UniversoAux (NIF);

-- 3.3. CRC 5G
DROP TABLE IF EXISTS #CRC5G
-- Verificar e excluir valores não numéricos da coluna idEnt
SELECT DISTINCT *
INTO #CRC5G
FROM (
    SELECT
        a1.idEnt,
        SUM(CASE
            WHEN LEFT(a1.dissTpInst, 3) IN ('005', '004') THEN 
                COALESCE(TRY_CAST(a1.dissMontTotal AS FLOAT), 0) * 0.03
            WHEN pr.PrazoResidual > 0 THEN 
                COALESCE(TRY_CAST(a1.dissMontTotal AS FLOAT), 0) / pr.PrazoResidual
            ELSE 0
        END) AS MM_Consolida,
        SUM(CASE
            WHEN pr.PrazoResidual > 0 AND a1.dissTpInst IN ('0130', '0140') THEN 
                COALESCE(TRY_CAST(a1.dissMontTotal AS FLOAT), 0) / pr.PrazoResidual
            ELSE 0
        END) AS MM_Consolida_Classicos,
        COUNT(CASE 
            WHEN TRY_CAST(a1.dissMontTotal AS FLOAT) > 0 THEN 1
            END) AS n_creditos
    FROM armada.dbo.CRC_DM_Disseminacao5G a1 WITH (NOLOCK)
    INNER JOIN #UniversoAux a ON a.NIF = a1.idEnt
    INNER JOIN (
        SELECT idEnt, MAX(dtInsercao) AS max_dtInsercao
        FROM armada.dbo.CRC_DM_Disseminacao5G
        WHERE ISNUMERIC(idEnt) = 1           -- EXCLUI VALORES NÃO NUMÉRICOS EM idEnt
        GROUP BY idEnt
    ) a2 ON a1.idEnt = a2.idEnt AND a1.dtInsercao = a2.max_dtInsercao
    CROSS APPLY (
        SELECT CASE
            WHEN TRY_CONVERT(DATE, a1.dissDtMatInst) IS NOT NULL
                 AND TRY_CONVERT(DATE, a1.dissDtMatInst) <> '9999-12-31'
            THEN DATEDIFF(MONTH, a1.dtRefDiss, TRY_CONVERT(DATE, a1.dissDtMatInst))
        END AS PrazoResidual
    ) pr
    WHERE ISNUMERIC(a1.idEnt) = 1               -- FILTRA idEnt APENAS NUMÉRICO
      AND (a1.dissTpInst IN ('0130', '0140') OR LEFT(a1.dissTpInst, 3) IN ('005', '004'))
      AND a1.dissTpResp IN ('001', '002')
      AND a1.dissNumDev = '1'
      AND ISNUMERIC(a1.dissMontTotal) = 1      -- FILTRA dissMontTotal PARA VALORES NUMÉRICOS
    GROUP BY a1.idEnt
) AS t
WHERE n_creditos > 1;


CREATE CLUSTERED INDEX IX_CRC5G ON #CRC5G (idEnt);

-- 3.4. CONSOLIDAÇÃO
SELECT
    a1.NIF,
    SUM(CASE WHEN IdProdutoFinanceiro IN (2,9,11,12) THEN 1 ELSE 0 END) AS N_Consolida,
    SUM(CASE WHEN IdProdutoFinanceiro IN (11,12) THEN 1 ELSE 0 END) AS N_ClassicoAuto,
    SUM(CASE WHEN IdProdutoFinanceiro IN (2,9,11,12) 
             THEN TRY_CAST(ValorAgregSaldo AS FLOAT) ELSE 0 END) AS DividaCRC
INTO #Consolida
FROM armada.dbo.IDH_DM_BPNovaCRC a1
INNER JOIN #UniversoAux a ON a.NIF = a1.NIF
WHERE IdSituacaoCredito = 1
  AND IdNivelResp = 1
GROUP BY a1.NIF;

CREATE CLUSTERED INDEX IX_Consolida ON #Consolida (NIF);

-- 3.5. REEMBOLSOS
SELECT
    a3.NIF,
    SUM(TRY_CAST(a1.MntReembolso AS FLOAT)) AS MntReembolso
INTO #Reembolsos
FROM armada.dbo.DM_Reembolsos a1
JOIN armada.dbo.DM_ClienteProcesso a3 ON a1.NumDossier = a3.NumDossier AND a3.Ordem = 1
WHERE a1.TipoReembolso = 'Total'
  AND TRY_CAST(a1.MntReembolso AS FLOAT) > 0
  AND a3.NIF <> 0
  AND LEFT(TRY_CAST(a1.NumDossier AS VARCHAR(20)),1) = '3'
GROUP BY a3.NIF;

CREATE CLUSTERED INDEX IX_Reembolsos ON #Reembolsos (NIF);

-- 3.6. UNIVERSO CCR FINAL
SELECT
    u.NIF,
    u.MesesTerminoMinimo,
    u.Segmento,
    CASE
        WHEN u.Segmento <> 'Terminados' THEN c.DividaCRC
        WHEN u.Segmento = 'Terminados' AND u.MesesTerminoMinimo > 12 THEN r.MntReembolso
        ELSE u.DividaCRC
    END AS DividaCRC,
    u.PrestMes,
    CASE WHEN c.N_Consolida > 1 THEN 1 ELSE 0 END AS MaisDoQue1CreditoConsolida,
    CASE WHEN c.N_ClassicoAuto = c.N_Consolida AND c.N_Consolida > 0 THEN 1 ELSE 0 END AS iSoAutoClassico,
    g.MM_Consolida,
    g.MM_Consolida_Classicos,
    g.n_creditos
INTO #UniversoCCR
FROM #UniversoAux u
LEFT JOIN #Consolida c ON u.NIF = c.NIF AND u.Segmento <> 'Terminados'
LEFT JOIN #Reembolsos r ON u.NIF = r.NIF AND u.Segmento = 'Terminados' AND u.MesesTerminoMinimo > 12
LEFT JOIN #CRC5G g ON TRY_CAST(u.NIF AS VARCHAR(50)) = TRY_CAST(g.idEnt AS VARCHAR(50))
WHERE (u.Segmento <> 'Terminados' AND c.N_Consolida > 1)
   OR (u.Segmento = 'Terminados' AND u.MesesTerminoMinimo <= 12 AND u.DividaCRC > 10000)
   OR (u.Segmento = 'Terminados' AND u.MesesTerminoMinimo > 12 AND r.MntReembolso > 10000);

CREATE CLUSTERED INDEX IX_UniversoCCR ON #UniversoCCR (NIF);

-- 3.7. CÁLCULO SOLVABILIDADE + DSTI
SELECT
    u.*,
    e.Ordem,
    CASE
        WHEN e.RendSolv IS NOT NULL AND e.DespesasSolv IS NOT NULL
        THEN TRY_CAST(e.RendSolv AS FLOAT) - 
             (TRY_CAST(e.DespesasSolv AS FLOAT) - COALESCE(u.MM_Consolida,0))
        ELSE 0
    END AS MensMaxSolvabilidade,
    CASE
        WHEN e.RendSolv IS NOT NULL AND e.PrestMes IS NOT NULL
        THEN TRY_CAST(e.RendSolv AS FLOAT) - 
             (TRY_CAST(e.PrestMes AS FLOAT) - COALESCE(u.MM_Consolida_Classicos,0))
        ELSE 0
    END AS MensMaxDSTI
INTO #Obra01
FROM #UniversoCCR u
INNER JOIN armada.dbo.DM_CampanhasElegibilidade e ON u.NIF = e.NIF
INNER JOIN #tab_max_dt_eleg eleg ON eleg.max_dt = e.IdDia
INNER JOIN #UniversoAux aux ON aux.NIF = e.NIF;

CREATE CLUSTERED INDEX IX_Obra01 ON #Obra01 (NIF);

-- 3.8. TABELA FINAL CCR
SELECT
    o.NIF,
    o.Ordem,
    o.Segmento,
    o.iSoAutoClassico,
    o.DividaCRC,
    o.PrestMes,
    o.MM_Consolida,
    o.MM_Consolida_Classicos,
    p.SegSMS,
    p.Montante,
    p.MontanteMax,
    p.Prazo,
    p.PrazoMax,
    p.TAN,
    p.TANMax,
    p.TAEG,
    p.TAEGMax,
    p.MM,
    p.MMMax,
    p.Seguro,
    p.SeguroMax,
    p.MTIC,
    p.MTICMax
INTO #TabelaFinal
FROM #Obra01 o
OUTER APPLY (
    SELECT TOP 1
        CASE
            WHEN o.Segmento <> 'Terminados' AND o.iSoAutoClassico = 1 
                 AND o.MM_Consolida - t.MM > 10 THEN 'SMSA'
            WHEN o.Segmento <> 'Terminados' THEN 'SMSB'
            WHEN o.Segmento = 'Terminados' AND o.MesesTerminoMinimo <= 12 THEN 'SMSC'
            WHEN o.Segmento = 'Terminados' AND o.MesesTerminoMinimo > 12 THEN 'SMSD'
        END AS SegSMS,
        t.Montante,
        t.Prazo,
        t.TAN,
        t.TAEG,
        t.MM,
        t.Seguro,
        t.MTIC,
        t.Montante AS MontanteMax,
        t.Prazo AS PrazoMax,
        t.TAN AS TANMax,
        t.TAEG AS TAEGMax,
        t.MM AS MMMax,
        t.Seguro AS SeguroMax,
        t.MTIC AS MTICMax
    FROM #proposta_comercial_CCR t
    WHERE o.DividaCRC > 500
      AND (
          (o.Segmento <> 'Terminados' AND o.MensMaxSolvabilidade > t.MM
           AND o.MensMaxDSTI > t.MM AND t.Montante > o.DividaCRC + 1000)
          OR (o.Segmento = 'Terminados' AND o.MesesTerminoMinimo <= 12 
              AND t.Montante > o.DividaCRC + 1000)
          OR (o.Segmento = 'Terminados' AND o.MesesTerminoMinimo > 12 
              AND t.Montante > o.DividaCRC)
      )
    ORDER BY CASE
        WHEN o.Segmento <> 'Terminados' AND o.iSoAutoClassico = 1 
        THEN t.Montante - (o.DividaCRC + 1000)
        ELSE t.Montante - o.DividaCRC
    END ASC
) p
WHERE p.Montante IS NOT NULL;

-- 3.9. RESULTADO FINAL CCR
SELECT 
    a.*,
    CASE WHEN SegSMS LIKE 'SMSA' THEN 1 ELSE 0 END AS iXS_CCR_SMSA,
    CASE WHEN SegSMS LIKE 'SMSB' THEN 1 ELSE 0 END AS iXS_CCR_SMSB,
    CASE WHEN SegSMS LIKE 'SMSC' THEN 1 ELSE 0 END AS iXS_CCR_SMSC,
    CASE WHEN SegSMS LIKE 'SMSD' THEN 1 ELSE 0 END AS iXS_CCR_SMSD
INTO #TabelaFinalCCR
FROM #TabelaFinal a;

/* =========================================================
   4. PMS - ELEGÍVEIS
========================================================= */
WITH BasePMS AS (
    SELECT
        cp.NIF,
        d.NumDossier,
        d.MntDivida,
        d.TxTAN,
        d.NumVencMensalidade,
        d.CodProdAlfa AS CodProduto,
        d.Plafond,
        d.MntMensalidade,
        r.DescricaoReferencia
    FROM armada.dbo.DM_Dossier d
    INNER JOIN armada.dbo.DM_ClienteProcesso cp ON cp.IdProcesso = d.IdProcesso
    LEFT JOIN armada.dbo.IDH_DM_PropostaIntegral pi ON pi.NumDossier = d.NumDossier
    LEFT JOIN armada.dbo.PAC_Dim_Referencias r ON r.IdReferencias = pi.IdTipoParceiro
    WHERE d.MntDivida > 0 
      AND d.TxTAN > 0 
      AND cp.NIF <> 0 
      AND d.NumVencMensalidade > 4
),
ContagensPMS AS (
    SELECT
        NIF,
        COUNT(DISTINCT NumDossier) AS QtdDossiersUniverso,
        COUNT(DISTINCT CASE WHEN CodProduto LIKE 'CA' AND NumVencMensalidade > 24 
                       THEN NumDossier END) AS QtdAutoElegivel,
        COUNT(DISTINCT CASE WHEN CodProduto <> 'CA' AND DescricaoReferencia <> 'Directo' 
                       AND NumVencMensalidade > 12 THEN NumDossier END) AS QtdParceriasElegivel
    FROM BasePMS
    GROUP BY NIF
),
Universo2PMS AS (
    SELECT
        cp.NIF,
        COUNT(DISTINCT d.NumDossier) AS QtdDossiersUniverso2
    FROM armada.dbo.DM_Dossier d
    INNER JOIN armada.dbo.DM_ClienteProcesso cp ON cp.IdProcesso = d.IdProcesso
    WHERE d.MntDivida > 0 AND cp.NIF <> 0
    GROUP BY cp.NIF
)
SELECT
    b.NIF,
    b.NumDossier,
    b.Plafond AS plafondmaisrecente,
    b.MntMensalidade,
    b.CodProduto
INTO #PMS
FROM BasePMS b
INNER JOIN ContagensPMS c ON c.NIF = b.NIF
INNER JOIN armada.dbo.DM_CampanhasElegibilidade_NIF n ON n.NIF = b.NIF
INNER JOIN Universo2PMS u2 ON u2.NIF = b.NIF
WHERE c.QtdDossiersUniverso = 1
  AND u2.QtdDossiersUniverso2 = 1
  AND (c.QtdAutoElegivel = 1 OR c.QtdParceriasElegivel = 1)
  AND n.iPMSPA_PM = 1
  AND b.CodProduto NOT IN ('RUC','CC');

/* =========================================================
   5. UNIVERSO BASE COMPLETO
========================================================= */
-- 5.1. CRUZAMENTO INICIAL
SELECT DISTINCT 
    eleg.*,
    CASE WHEN s.NIF IS NOT NULL THEN MensMaxSolvabilidadeNEW 
         ELSE eleg.MensMaxSolvabilidade END AS MensMaxSolvabilidadeNEW,
    CASE WHEN s.NIF IS NOT NULL THEN excl_solvNEW 
         ELSE excl_solv END AS excl_solvNEW
INTO #Cruzamento
FROM armada.dbo.DM_CampanhasElegibilidade eleg
INNER JOIN #tab_max_dt_eleg g ON g.max_dt = eleg.IdDia
INNER JOIN armada.dbo.DM_CampanhasElegibilidade_NIF nif ON nif.NIF = eleg.NIF
INNER JOIN armada.dbo.DM_Dossier d ON d.NumDossier = eleg.NumDossierOrigem
LEFT JOIN #SemSolvabilidade s ON s.NIF = eleg.NIF
LEFT JOIN #PMS p ON p.NIF = eleg.NIF
WHERE NOT (AtividadeCredito LIKE 'Inativo' AND MesesInatividadeMinimo > 120);

-- 5.2. UNIVERSO COM FLAGS
SELECT DISTINCT
    eleg.*,
    LEFT(DtAbertura, 4) AS AnoGeracao,
    -- iPCD_PA_PM
    CASE WHEN eleg.excl_score = 1 OR eleg.excl_Financiavel = 1 OR eleg.excl_ENI = 1 
              OR eleg.excl_removidos = 1 OR eleg.excl_solvNEW = 1 OR eleg.excl_idade = 1
              OR eleg.excl_analise = 1 OR eleg.excl_ultRecusa = 1 OR eleg.excl_UltAbertura = 1
              OR eleg.excl_reclamacao = 1 OR eleg.excl_RequerInvestigacao = 1 
              OR eleg.excl_IncidentesBancarios = 1 OR eleg.excl_SocioGerente = 1
              OR eleg.excl_prof_instavel = 1
              OR (AtividadeCredito LIKE 'Inativo' AND MesesInatividadeMinimo > 120)
         THEN 0 ELSE 1 END AS iPCD_PA_PM,
    -- iPCD_PA_GM
    CASE WHEN eleg.excl_score = 1 OR eleg.excl_Financiavel = 1 OR eleg.excl_ENI = 1 
              OR eleg.excl_removidos = 1 OR eleg.excl_solvNEW = 1 OR eleg.excl_idade = 1
              OR eleg.excl_analise = 1 OR eleg.excl_ultRecusa = 1 OR eleg.excl_UltAbertura = 1
              OR eleg.excl_DSTI = 1 OR eleg.excl_DO = 1 OR eleg.excl_Rend = 1
              OR eleg.excl_reclamacao = 1 OR eleg.excl_RequerInvestigacao = 1 
              OR eleg.excl_IncidentesBancarios = 1 OR eleg.excl_SocioGerente = 1
              OR eleg.excl_prof_instavel = 1
              OR (AtividadeCredito LIKE 'Inativo' AND MesesInatividadeMinimo > 120)
         THEN 0 ELSE 1 END AS iPCD_PA_GM,
    -- iRUC_PA
    CASE WHEN eleg.excl_revolving = 1 OR eleg.excl_score = 1 OR eleg.excl_Financiavel = 1 
              OR eleg.excl_ENI = 1 OR eleg.excl_removidos = 1 OR eleg.excl_solvNEW = 1
              OR eleg.excl_idade = 1 OR eleg.excl_analise = 1 OR eleg.excl_ultRecusa = 1
              OR eleg.excl_UltAbertura = 1 OR eleg.excl_DO = 1 OR eleg.excl_Rend = 1
              OR eleg.excl_reclamacao = 1 OR eleg.excl_RequerInvestigacao = 1
              OR eleg.excl_IncidentesBancarios = 1 OR eleg.excl_SocioGerente = 1
              OR eleg.excl_prof_instavel = 1
              OR (AtividadeCredito LIKE 'Inativo' AND MesesInatividadeMinimo > 120)
         THEN 0 ELSE 1 END AS iRUC_PA,
    iCCR, iAP, iAD, iAI, ScoreApetencia,
    -- iPMS_PA_PM
    CASE WHEN (eleg.excl_score = 1 OR eleg.excl_Financiavel = 1 OR eleg.excl_ENI = 1 
               OR eleg.excl_removidos = 1 OR eleg.excl_solvNEW = 1 OR eleg.excl_idade = 1
               OR eleg.excl_analise = 1 OR eleg.excl_ultRecusa = 1 OR eleg.excl_UltAbertura = 1
               OR eleg.excl_reclamacao = 1 OR eleg.excl_RequerInvestigacao = 1 
               OR eleg.excl_IncidentesBancarios = 1 OR eleg.excl_SocioGerente = 1
               OR eleg.excl_prof_instavel = 1
               OR (AtividadeCredito LIKE 'Inativo' AND MesesInatividadeMinimo > 120))
               AND p.NIF IS NOT NULL
         THEN 0 ELSE 1 END AS iPMS_PA_PM,
    -- iPCD_NPA
    CASE WHEN eleg.excl_Financiavel = 1 OR eleg.excl_ENI = 1 OR eleg.excl_removidos = 1 
              OR TxSolvabilidade > 100 OR eleg.excl_idade = 1 OR eleg.MesesUltAbertura < 4
              OR eleg.excl_analise = 1 OR eleg.excl_ultRecusa = 1 
              OR eleg.excl_IncidentesBancarios = 1 OR eleg.excl_Nacionalidade = 1
              OR eleg.excl_reclamacao = 1 OR eleg.excl_tiporesidencia = 1
              OR eleg.excl_prof_instavel = 1 OR AtividadeCredito = 'Terminado'
              OR (AtividadeCredito LIKE 'Inativo' AND MesesInatividadeMinimo > 120)
         THEN 0 ELSE 1 END AS iPCD_NPA,
    -- iRUC_NPA
    CASE WHEN eleg.excl_Financiavel = 1 OR eleg.excl_ENI = 1 OR eleg.excl_removidos = 1 
              OR TxSolvabilidade > 100 OR eleg.excl_idade = 1 OR eleg.excl_analise = 1
              OR eleg.MesesUltAbertura < 4 OR eleg.excl_IncidentesBancarios = 1
              OR eleg.excl_DO = 1 OR eleg.excl_Nacionalidade = 1 OR eleg.excl_reclamacao = 1
              OR eleg.excl_tiporesidencia = 1 OR eleg.excl_prof_instavel = 1
              OR eleg.excl_revolving = 1 OR AtividadeCredito = 'Terminado'
              OR (AtividadeCredito LIKE 'Inativo' AND MesesInatividadeMinimo > 120)
         THEN 0 ELSE 1 END AS iRUC_NPA,
    -- iTerminadosRUC
    CASE WHEN eleg.excl_Financiavel = 1 OR eleg.excl_ENI = 1 OR eleg.excl_removidos = 1 
              OR TxSolvabilidade > 100 OR eleg.excl_IncidentesBancarios = 1
              OR eleg.excl_idade = 1 OR eleg.excl_analise = 1 OR eleg.MesesUltAbertura < 4
              OR eleg.excl_revolving = 1 OR eleg.excl_Nacionalidade = 1
              OR eleg.excl_reclamacao = 1 OR eleg.excl_tiporesidencia = 1
              OR eleg.excl_prof_instavel = 1 OR AtividadeCredito <> 'Terminado'
         THEN 0 ELSE 1 END AS iTerminadosRUC,
    -- iTerminadosPCD
    CASE WHEN eleg.excl_Financiavel = 1 OR eleg.MesesUltAbertura < 4 OR eleg.excl_ENI = 1 
              OR eleg.excl_removidos = 1 OR TxSolvabilidade > 100 OR eleg.excl_idade = 1
              OR eleg.excl_analise = 1 OR eleg.excl_ultRecusa = 1 
              OR eleg.excl_IncidentesBancarios = 1 OR eleg.excl_Nacionalidade = 1
              OR eleg.excl_reclamacao = 1 OR eleg.excl_tiporesidencia = 1
              OR eleg.excl_prof_instavel = 1 OR AtividadeCredito <> 'Terminado'
         THEN 0 ELSE 1 END AS iTerminadosPCD
INTO #Universo
FROM #Cruzamento eleg
INNER JOIN #tab_max_dt_eleg g ON g.max_dt = eleg.IdDia
INNER JOIN armada.dbo.DM_CampanhasElegibilidade_NIF nif ON nif.NIF = eleg.NIF
INNER JOIN armada.dbo.DM_Dossier d ON d.NumDossier = eleg.NumDossierOrigem
LEFT JOIN #SemSolvabilidade s ON s.NIF = eleg.NIF
LEFT JOIN #PMS p ON p.NIF = eleg.NIF
WHERE NOT (AtividadeCredito LIKE 'Inativo' AND MesesInatividadeMinimo > 120);

/* =========================================================
   6. INFORMAÇÕES DE CAMPANHA E PRESSÃO COMERCIAL
========================================================= */
WITH CTE_UltimaCampanha AS (
    SELECT
        ec.NumContribuinte AS NIF,
        cd.Descricao,
                        CASE WHEN cd.Descricao LIKE '%R+1%' THEN 'iXS_RUC_R+1'
              WHEN cd.Descricao LIKE '%TOP%' AND cd.IdProduto = 25 THEN 'iXS_PCDPMTOP'
        WHEN cd.Descricao LIKE '%INA%' AND cd.IdProduto = 25 THEN 'iXS_PCD_Inativos_1T'
        WHEN cd.Descricao LIKE '%LAR%' AND cd.IdProduto = 25 THEN 'iXS_PCDPMHabit'
        WHEN cd.Descricao LIKE '%PARCERIAS%' AND cd.IdProduto = 25 THEN 'iXS_PCDPM_Parcerias'
        WHEN (cd.Descricao LIKE '%Auto%' or cd.Descricao LIKE '%MOTO%') and cd.IdProduto = 25 THEN 'iXS_PCD_Auto'
        WHEN cd.Descricao LIKE '%PLAFONDM%' AND cd.IdProduto = 23 THEN 'iXS_RUC_PlafondMinimo'

        WHEN cd.Descricao Like 'CSPCDPA_APT_DIG_JAN23' or cd.Descricao Like 'CSPCDPA_APT_DIG_DEZ22' or cd.Descricao LIKE 'CSPCDPA_APT_DIG' or (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%1TIT%' or cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%' or cd.Descricao LIKE 'CSPCDPA_BYSIDE') AND cd.IdProduto = 25 AND cd.Tipo = 4615) THEN 'iXS_PCDPM_1T'
        WHEN cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%2TIT%' or cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%') AND cd.IdProduto = 25 AND cd.Tipo = 4615 THEN 'iXS_PCDPM_2T'

        WHEN (cd.Descricao LIKE 'CSRUCAUTO_CV') or cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%1TIT%' or cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.IdProduto = 23 AND cd.Tipo = 4615 THEN 'iXS_RUC_1T'
        WHEN cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%2TIT%' or cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%') AND cd.IdProduto = 23 AND cd.Tipo = 4615 THEN 'iXS_RUC_2T'

        WHEN (cd.Descricao LIKE '%CPAY%' AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 25) or cd.Descricao LIKE '%COMBO_CPAY%' or cd.Descricao LIKE 'CSPCD_PAY' or cd.Descricao LIKE 'CSPCD_CPAY'  or cd.Descricao LIKE '%CSPCDCPAY_VOUCHER%'  THEN 'iXS_PCD_NPA_CPay'
        WHEN (cd.Descricao LIKE '%CPAY%' AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 23) or cd.Descricao LIKE '%RUCCPAY%' or cd.Descricao LIKE 'CSRUC_CPAY' THEN 'iXS_RUC_NPA_CPay'

        WHEN (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 25) or cd.Descricao LIKE 'CSPCDSITE_NPA' or cd.Descricao LIKE 'CSPCD_NPA' THEN 'iXS_PCD_NPA_1T'
        WHEN cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%') AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 25 THEN 'iXS_PCD_NPA_2T'
        WHEN (cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 23) or cd.Descricao LIKE 'CSRUCSITE_NPA' or cd.Descricao LIKE 'CSRUC_NPA' THEN 'iXS_RUC_NPA_1T'
        WHEN cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.IdProduto = 23 THEN 'iXS_RUC_NPA_2T'

        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%PA%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_PA_1T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.Descricao LIKE '%PA%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_PA_2T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%CAPT%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_Captacao_1T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.Descricao LIKE '%CAPT%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_Captacao_2T'
        WHEN (cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.IdProduto = 25) or cd.Descricao LIKE 'CSTERM' THEN 'iXS_Terminados_NPA_1T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.IdProduto = 25 THEN 'iXS_Terminados_NPA_2T'

        WHEN cd.IdProduto = 28 THEN 'iXS_PMS'
        WHEN (cd.Descricao LIKE '%SEG_A%' AND cd.IdProduto = 22) or cd.Descricao LIKE 'CSCCR' or cd.Descricao LIKE '%CCR_SMSA%' or cd.Descricao LIKE 'CSCCR_A' or cd.Descricao LIKE '%CCR_SMSA_C2C%' or cd.Descricao LIKE 'CCR_Geral_AtivCofidis' or cd.Descricao LIKE 'CSCCR_LOJ' THEN 'iXS_CCR_SMSA'
        WHEN (cd.Descricao LIKE '%SEG_B%' or cd.Descricao LIKE '%CCR_SMSB%' or cd.Descricao LIKE 'CSCCR_B') AND cd.IdProduto = 22 THEN 'iXS_CCR_SMSB'
        WHEN (cd.Descricao LIKE '%SEG_C%' or cd.Descricao LIKE '%CCR_SMSC%' or cd.Descricao LIKE 'CSCCR_C') AND cd.IdProduto = 22 THEN 'iXS_CCR_SMSC'
        WHEN (cd.Descricao LIKE '%SEG_D%' or cd.Descricao LIKE '%CCR_SMSD%' or cd.Descricao LIKE 'CSCCR_D') AND cd.IdProduto = 22 THEN 'iXS_CCR_SMSD'
        WHEN cd.Descricao LIKE '%T3%' AND cd.IdProduto = 25 THEN 'iXS_RUC_NPA_3T'
        WHEN (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%SITE%') AND cd.IdProduto = 25 AND cd.Tipo = 4615) or (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%SUD%') AND cd.IdProduto = 25 AND cd.Tipo = 4615) THEN 'iXS_PCDPM_1T'
        WHEN (cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%SITE%') AND cd.IdProduto = 23 AND cd.Tipo = 4615) or cd.Descricao LIKE '%CSRUC_SITE%' or (cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%SUD%') AND cd.IdProduto = 23 AND cd.Tipo = 4615) or cd.Descricao LIKE '%CSRUCPA%' THEN 'iXS_RUC_1T'
        ELSE NULL
        END as iTipoCampanha,
        p.CodProduto AS UltimoProdutoProposto,
        ec.DtCarregamento,
        ec.MontanteProposto,
        CASE 
            WHEN cd.Tipo = 4615 THEN 'PA'
            WHEN cd.Tipo = 528 THEN 'NPA'
            ELSE 'NUNCA RECEBEU' 
        END AS iCampanha,
        ROW_NUMBER() OVER (PARTITION BY ec.NumContribuinte ORDER BY ec.DtCarregamento DESC) AS RN
    FROM armada.dbo.DM_EnviosCampanhas ec
    INNER JOIN armada.dbo.DM_CampanhasDefinicao cd ON cd.IdCampanha = ec.IdCampanha
    LEFT JOIN armada.dbo.IDH_Dim_Produto p ON p.IdProduto = cd.IdProduto
    WHERE cd.Tipo IN (4615, 528) and cd.Descricao NOT LIKE '%ELEG%' and cd.Descricao NOT LIKE '%PRM%'
),
CTE_Pressao AS (
    SELECT
        ec.NumContribuinte AS NIF,
        COUNT(DISTINCT ec.IdCampanha) AS PressaoComercial
    FROM armada.dbo.DM_EnviosCampanhas ec
    INNER JOIN armada.dbo.DM_CampanhasDefinicao cd ON cd.IdCampanha = ec.IdCampanha
    WHERE ec.DtCarregamento >= DATEADD(YEAR, -1, GETDATE())
      AND cd.Tipo IN (4615, 528)
    GROUP BY ec.NumContribuinte
),
CTE_RUCPlafond AS (
    SELECT
        ec.NumContribuinte AS NIF,
        COUNT(DISTINCT ec.IdCampanha) AS QtdRUCPlafond
    FROM armada.dbo.DM_EnviosCampanhas ec
    INNER JOIN armada.dbo.DM_CampanhasDefinicao cd ON cd.IdCampanha = ec.IdCampanha
    WHERE ec.DtCarregamento >= DATEADD(MONTH, -3, GETDATE())
      AND cd.Descricao LIKE 'CSRUC_PLAFONDM_CPAY'
    GROUP BY ec.NumContribuinte
)
SELECT
    u.*,
    COALESCE(c.Descricao, 'Sem Campanha') AS DescricaoUltimaCampanha,
    COALESCE(c.iTipoCampanha, 'Sem Campanha') AS TipoUltimaCampanha,
    COALESCE(c.iCampanha, 'Sem Campanha') AS TipologiaUltimaCampanha,
    c.MontanteProposto AS MontanteUltimaCampanha,
    c.UltimoProdutoProposto AS ProdutoUltimaCampanha,
    FORMAT(c.DtCarregamento, 'yyyyMM') AS IdMesUltimaCampanha,
    pc.PressaoComercial,
    CASE WHEN pp.QtdRUCPlafond < 2 THEN 1 ELSE 0 END AS iPressaoPlafondMinimo
INTO #BaseConsolidada
FROM #Universo u
LEFT JOIN CTE_UltimaCampanha c ON c.NIF = u.NIF AND c.RN = 1
LEFT JOIN CTE_Pressao pc ON pc.NIF = u.NIF
LEFT JOIN CTE_RUCPlafond pp ON pp.NIF = u.NIF;

/* =========================================================
   7. FLAGS ADICIONAIS (LAR, AUTO, MOTO, PLAFOND)
========================================================= */
SELECT
    b.*,
    pm.MensalidadeCofidisPay,
    CASE WHEN hab.NIF IS NOT NULL OR lar.NIF IS NOT NULL THEN 1 ELSE 0 END AS iLar,
    CASE WHEN cli.IdCliente IS NOT NULL THEN 1 ELSE 0 END AS iAuto,
    CASE WHEN moto.NIF IS NOT NULL THEN 1 ELSE 0 END AS iMoto,
    CASE WHEN pm.NIF IS NOT NULL
          AND SubcanalNegocioAtual LIKE 'CofidisPay'
          AND (AtividadeCredito LIKE 'Ativo' OR (AtividadeCredito < 'Terminado' AND MesesTerminoMinimo < 2))
          AND iPressaoPlafondMinimo = 1
         THEN 1 ELSE 0 END AS iPlafondMinimo,
    CASE WHEN DataFimMotivo >= CONVERT(VARCHAR(8), DATEADD(DAY, -7, CAST(GETDATE() AS DATE)), 112) THEN 1 ELSE 0 end as iQuarentenaFinanciabilidade  
INTO #BaseFinal
FROM #BaseConsolidada b
LEFT JOIN (
    SELECT CAST(idEnt AS BIGINT) AS NIF
    FROM armada.dbo.CRC_DM_Disseminacao5G
    WHERE dissTpInst = '0110' AND CAST(dissMontTotal AS FLOAT) > 0
    GROUP BY idEnt
) hab ON hab.NIF = b.NIF
LEFT JOIN (
    SELECT DISTINCT cp.NIF
    FROM armada.dbo.IDH_DM_PropostaIntegral pi
    INNER JOIN armada.dbo.DM_ClienteProcesso cp ON pi.IdProcesso = cp.IdProcesso
    INNER JOIN armada.dbo.PAC_Dim_Referencias pref ON pi.IdBemAFinanciar = pref.IdReferencias
    WHERE pi.Canal = 'Parcerias'
      AND pref.IdReferencias IN (
        104795,104648,104447,104793,104796,104797,104802,104804,
        104641,104456,104446,104448,104803,104808,104806,104646,
        104647,104791,104629,104786,104792,104798
      )
) lar ON lar.NIF = b.NIF
LEFT JOIN armada.dbo.IDH_DM_Cliente cli ON cli.IdCliente = b.IdCliente
       AND cli.IdDataNascimento <= CAST(FORMAT(DATEADD(YEAR, -25, GETDATE()), 'yyyyMMdd') AS INT)
LEFT JOIN (
    SELECT DISTINCT cp.NIF
    FROM armada.dbo.IDH_DM_PropostaIntegral pi
    INNER JOIN armada.dbo.DM_ClienteProcesso cp ON cp.IdProcesso = pi.IdProcesso
    WHERE pi.IdFinalidadeComercial = 19
      AND pi.DtPedido < CAST(FORMAT(DATEADD(YEAR, -3, GETDATE()), 'yyyyMMdd') AS INT)
) moto ON moto.NIF = b.NIF
LEFT JOIN (
    SELECT
        cp.NIF,
        MAX(MntMensalidadeCnt) AS MensalidadeCofidisPay
    FROM armada.dbo.DM_Dossier d
    INNER JOIN armada.dbo.DM_ClienteProcesso cp ON cp.IdProcesso = d.IdProcesso
    WHERE CodProdAlfa LIKE 'SPF'
      AND (LEFT(DtFimContrato, 6) = FORMAT(GETDATE(), 'yyyyMM')
           OR (LEFT(DtUltPagamento, 6) > LEFT(CONVERT(VARCHAR(8), DATEADD(MONTH, -2, CAST(GETDATE() AS DATE)), 112), 6) 
               AND MntDivida <= 0))
    GROUP BY cp.NIF
) pm ON pm.NIF = b.NIF
LEFT JOIN (SELECT IdTitular as IdCliente, MAX(DataFimMotivo) as DataFimMotivo
FROM armada.dbo.DM_CriteriosNFinanciavelTitular
WHERE DataFimMotivo <> 20301231 and CodMotivo IN ('1','3','4','6','8','10','12','20','26','27A','27B','27C','27D','27E','27F','27G','27H','28','10A','13')
GROUP BY IdTitular) as nf 
on nf.IdCliente = b.IdCliente;


/* =========================================================
   8. PROPOSTAS COMERCIAIS
========================================================= */
-- 8.1. RUC
WITH CTE_RUC AS (
    SELECT a.*, d.Descricao,
           ROW_NUMBER() OVER (PARTITION BY AMOUNT ORDER BY OfferTarificationId DESC, a.DeadlineWithInsurance DESC) AS rn
    FROM armada.dbo.ST_DM_CondicoesSimulacao a
    LEFT JOIN armada.dbo.DM_Oferta b ON a.idoffer = b.idOferta
    LEFT JOIN armada.dbo.IDH_Dim_Produto c ON b.Produto = c.CodProdutoDAH
    LEFT JOIN armada.dbo.IDH_Dim_Finalidade d ON a.PURPOSEID = d.IDFinalidade
    WHERE c.CodProduto = 'RUC' AND a.AMOUNT IN (1000,1500,2000,3000,4000)
)
SELECT * INTO #proposta_comercial_RUC FROM CTE_RUC WHERE rn = 1;

-- 8.2. PCD Intermediários
WITH CTE_PCD_Intermedios AS (
    SELECT a.*, d.Descricao,
           ROW_NUMBER() OVER (PARTITION BY AMOUNT ORDER BY OfferTarificationId DESC, a.DeadlineWithInsurance DESC) AS rn
    FROM armada.dbo.ST_DM_CondicoesSimulacao a
    LEFT JOIN armada.dbo.DM_Oferta b ON a.idoffer = b.idOferta
    LEFT JOIN armada.dbo.IDH_Dim_Produto c ON b.Produto = c.CodProdutoDAH
    LEFT JOIN armada.dbo.IDH_Dim_Finalidade d ON a.PURPOSEID = d.IDFinalidade
    WHERE a.AMOUNT IN (9000,10000,11000,12000,13000,14000,15000)
      AND DEADLINEWITHINSURANCE = 84 AND IDFinalidade = 6
)
SELECT * INTO #proposta_comercial_PCD_Intermedios FROM CTE_PCD_Intermedios WHERE rn = 1;

-- 8.3. PCD
WITH CTE_PCD AS (
    SELECT a.*, d.Descricao,
           ROW_NUMBER() OVER (PARTITION BY AMOUNT ORDER BY OfferTarificationId DESC, a.DeadlineWithInsurance DESC) AS rn
    FROM armada.dbo.ST_DM_CondicoesSimulacao a
    LEFT JOIN armada.dbo.DM_Oferta b ON a.idoffer = b.idOferta
    LEFT JOIN armada.dbo.IDH_Dim_Produto c ON b.Produto = c.CodProdutoDAH
    LEFT JOIN armada.dbo.IDH_Dim_Finalidade d ON a.PURPOSEID = d.IDFinalidade
    WHERE a.AMOUNT IN (2500,3000,3500,4000,4500,5000,6000,6500,7000,7500,8000)
      AND DEADLINEWITHINSURANCE = 84 AND IDFinalidade = 6
)
SELECT * INTO #proposta_comercial_PCD FROM CTE_PCD WHERE rn = 1;

-- 8.4. PCD Auto
WITH CTE_PCDAuto AS (
    SELECT a.*, d.Descricao,
           ROW_NUMBER() OVER (PARTITION BY AMOUNT ORDER BY OfferTarificationId DESC, a.DeadlineWithInsurance DESC) AS rn
    FROM armada.dbo.ST_DM_CondicoesSimulacao a
    LEFT JOIN armada.dbo.DM_Oferta b ON a.idoffer = b.idOferta
    LEFT JOIN armada.dbo.IDH_Dim_Produto c ON b.Produto = c.CodProdutoDAH
    LEFT JOIN armada.dbo.IDH_Dim_Finalidade d ON a.PURPOSEID = d.IDFinalidade
    WHERE ((AMOUNT IN (8000,9000) AND DEADLINEWITHINSURANCE = 96)
        OR (AMOUNT IN (10000,11000,12000,13000,14000,15000,16000) AND DEADLINEWITHINSURANCE = 120)
        OR (AMOUNT IN (5000,6000,7000) AND DEADLINEWITHINSURANCE = 84))
      AND IDFinalidade = 2
)
SELECT * INTO #proposta_comercial_PCDAuto FROM CTE_PCDAuto WHERE rn = 1;

SELECT DISTINCT
CAST(a.AMOUNT AS INT) AS MONTANTE,
CAST(a.TAN AS DECIMAL(10,2)) AS TAN,
CAST(a.TAEG AS DECIMAL(10,1)) AS TAEG,
CAST(a.DeadlineWithInsurance AS INT) AS Prazo,
CAST(a.MONTHLYPAYWITHOUTINSURANCE AS DECIMAL(10,2)) AS MM,
CAST(a.Insurance AS DECIMAL(10,2)) AS Seguro,
CAST(a.MTIC AS DECIMAL(10,2)) AS MTIC
INTO #proposta_comercial_PMS
FROM DataExperimental_DICV.dbo.TabelaFinanceira_PMS a;


-- 8.6. Cálculo PMS
WITH base_calculo_pms AS (
    SELECT
        a.NIF,
        a.RendSolv,
        a.DespesasSolv,
        a.PrestMes,
        a.PrestMesCofidis,
        a.MntDivida AS encours_cliente,
        p.plafondmaisrecente,
        b.MONTANTE,
        b.PRAZO,
        b.TAN,
        b.TAEG,
        b.MM,
        b.Seguro,
        b.MTIC,
        (a.RendSolv - (a.DespesasSolv - a.PrestMesCofidis)) AS MM_Max,
        (b.MM - a.PrestMes) AS aumento_mensalidade,
        (b.MONTANTE - a.MntDivida) AS liquidez_adicional
    FROM armada.dbo.DM_CampanhasElegibilidade a
    INNER JOIN #tab_max_dt_eleg e ON e.max_dt = a.IdDia
    INNER JOIN #PMS p ON p.NIF = a.NIF
    CROSS JOIN #proposta_comercial_PMS b
),
regras_alvo_pms AS (
    SELECT *,
        CASE
            WHEN encours_cliente <= 4000 THEN encours_cliente + 2000
            WHEN encours_cliente > 4000 AND encours_cliente <= 8000 THEN encours_cliente * 2
            WHEN encours_cliente > 8000 THEN encours_cliente - MONTANTE
        END AS montante_alvo
    FROM base_calculo_pms
),
metricas_pms AS (
    SELECT *,
        ABS(MONTANTE - montante_alvo) AS dif_montante,
        (liquidez_adicional - 1000) AS dif_liquidez_min,
        (liquidez_adicional - (0.20 * encours_cliente)) AS dif_liquidez_pct,
        (MONTANTE - (2 * plafondmaisrecente)) AS dif_plafond,
        (MM_Max - MM) AS dif_MM
    FROM regras_alvo_pms
),
filtrada_pms AS (
    SELECT *
    FROM metricas_pms
    WHERE dif_MM > 0
      AND aumento_mensalidade < 80
      AND dif_liquidez_min >= 0
      AND dif_liquidez_pct >= 0
      AND dif_plafond >= 0
)
SELECT
    NIF, MONTANTE, PRAZO, TAN, TAEG, MM, Seguro, MTIC
INTO #ElegiveisPMS
FROM (
    SELECT *,
           ROW_NUMBER() OVER (PARTITION BY NIF ORDER BY dif_montante ASC, MONTANTE DESC) AS rn_final
    FROM filtrada_pms
) t
WHERE rn_final = 1;

/* =========================================================
   9. MATCHING FINANCEIRO
========================================================= */
SELECT
    bf.*,
    (SELECT TOP 1 pc.Amount
     FROM #proposta_comercial_RUC pc
     WHERE pc.MonthlyPayWithoutInsurance IS NOT NULL
       AND bf.MensMaxSolvabilidadeNEW >= pc.MonthlyPayWithoutInsurance
     ORDER BY pc.Amount DESC) AS Melhor_Match_RUC,
    (SELECT TOP 1 pc.Amount
     FROM #proposta_comercial_PCD pc
     WHERE pc.MonthlyPayWithoutInsurance IS NOT NULL
       AND bf.MensMaxSolvabilidadeNEW >= pc.MonthlyPayWithoutInsurance
     ORDER BY pc.Amount DESC) AS Melhor_Match_PCD,
    (SELECT TOP 1 pc.Amount
     FROM #proposta_comercial_PCD_Intermedios pc
     WHERE pc.MonthlyPayWithoutInsurance IS NOT NULL
       AND bf.MensMaxSolvabilidadeNEW >= pc.MonthlyPayWithoutInsurance
     ORDER BY pc.Amount DESC) AS Melhor_Match_PCD_MI,
    (SELECT TOP 1 pc.Amount
     FROM #proposta_comercial_PCDAuto pc
     WHERE pc.MonthlyPayWithoutInsurance IS NOT NULL
       AND bf.MensMaxSolvabilidadeNEW >= pc.MonthlyPayWithoutInsurance
     ORDER BY pc.Amount DESC) AS Melhor_Match_PCDAuto,
    (SELECT TOP 1 pc.Amount
     FROM #proposta_comercial_RUC pc
     WHERE pc.MonthlyPayWithoutInsurance IS NOT NULL
       AND bf.MensalidadeCofidisPay >= pc.MonthlyPayWithoutInsurance
     ORDER BY pc.Amount DESC) AS Melhor_Match_RUCPlafondMinimo,
     c.Montante as Melhor_Match_CCD,
     p.MONTANTE as Melhor_Match_PMS
INTO #Resultado_Financeiro
FROM #BaseFinal bf
LEFT JOIN #TabelaFinalCCR c on c.NIF = bf.NIF 
LEFT JOIN #ElegiveisPMS p on p.NIF = bf.NIF;

/* =========================================================
   10. BASE DE ENVIO FINAL
========================================================= */
WITH BasePCDPMTOP AS (
    SELECT DISTINCT
        NIF,
        ROW_NUMBER() OVER (ORDER BY ScoreApetencia DESC) AS rn_PCDPMTOP
    FROM #Resultado_Financeiro
    WHERE iPCD_PA_PM = 1
      AND Melhor_Match_PCD IS NOT NULL
      AND Ordem = 1
      AND (iOptOut_Email = 0 OR iOptOut_SMS = 0)
      AND AtividadeCredito = 'Ativo'
      AND ScoreApetencia < 0
)
SELECT DISTINCT
    rf.NIF,
    rf.IdCliente,
    NIF.PrimeiroNome as PrimNome,
    CASE WHEN nif.Email LIKE '' THEN NULL
         WHEN iOptOut_Email = 0 THEN NIF.Email
         ELSE NULL END as Email,
    CASE WHEN nif.Telemovel LIKE '' THEN NULL
         WHEN nif.Telemovel = 0 THEN NULL
         WHEN LEN(nif.Telemovel) = 9 AND iOptOut_SMS = 0 THEN nif.Telemovel
         ELSE NULL END AS telemovel,
    CASE
        WHEN nif.Loja = 'SEDE' THEN 'LISBOA'
        WHEN nif.Loja = 'FARO' THEN 'SETUBAL'
        WHEN nif.Loja = 'CASTELO BRANCO' THEN 'COIMBRA'
        WHEN nif.Loja = 'EVORA' THEN 'SETUBAL'
        WHEN nif.Loja = 'MIRANDELA' THEN 'VISEU'
        WHEN nif.Loja = 'SINTRA' THEN 'LISBOA'
        WHEN nif.Loja = 'VIANA DO CASTELO' THEN 'BRAGA'
        WHEN nif.Loja = 'TERCEIRA' THEN 'AÇORES'
        WHEN nif.Loja like '' THEN 'LISBOA'
        ELSE nif.Loja
    END as NomeLoja,
    IdMesUltimaCampanha,
    TipoUltimaCampanha,
    TipologiaUltimaCampanha,
    ProdutoUltimaCampanha,
    Melhor_Match_PCD,
    Melhor_Match_PCD_MI,
    Melhor_Match_RUC,
    Melhor_Match_PCDAuto,
    Melhor_Match_RUCPlafondMinimo,
    c.Montante as Melhor_Match_CCR,
    iQuarentenaFinanciabilidade,
    -- PCD AUTO
    CASE WHEN iAuto = 1 AND (iPCD_PA_PM = 1 OR iPCD_PA_GM = 1)
              AND Melhor_Match_PCD IS NOT NULL AND Ordem = 1
              AND AtividadeCredito = 'Ativo'
         THEN 1 ELSE 0 END AS iXS_PCD_Auto,
    -- PCDPM HABITACIONAL
    CASE WHEN iLar = 1 AND iPCD_PA_PM = 1
              AND Melhor_Match_PCD IS NOT NULL AND Ordem = 1
              AND AtividadeCredito = 'Ativo'
         THEN 1 ELSE 0 END AS iXS_PCDPMHabit,
    -- PCDPM TOP
    CASE WHEN iPCD_PA_PM = 1 AND Melhor_Match_PCD IS NOT NULL
              AND Ordem = 1 AND AtividadeCredito = 'Ativo'
              AND b.rn_PCDPMTOP <= 25000
         THEN 1 ELSE 0 END AS iXS_PCDPMTOP,
    -- PCDPM PARCERIAS
    CASE WHEN iPCD_PA_PM = 1 AND Melhor_Match_PCD IS NOT NULL
              AND CanalNegocioAtual = 'Parcerias'
              AND Ordem = 1 AND AtividadeCredito = 'Ativo'
         THEN 1 ELSE 0 END AS iXS_PCDPM_Parcerias,
    -- PCDPM 1T e 2T
    CASE WHEN iPCD_PA_PM = 1 AND Melhor_Match_PCD IS NOT NULL
              AND Ordem = 1 AND AtividadeCredito = 'Ativo'
         THEN 1 ELSE 0 END AS iXS_PCDPM_1T,
    CASE WHEN iPCD_PA_PM = 1 AND Melhor_Match_PCD IS NOT NULL
              AND Ordem = 2 AND AtividadeCredito = 'Ativo'
         THEN 1 ELSE 0 END AS iXS_PCDPM_2T,
    -- RUC 1T e 2T
    CASE WHEN iRUC_PA = 1 AND Melhor_Match_RUC IS NOT NULL
              AND Ordem = 1 AND AtividadeCredito = 'Ativo'
         THEN 1 ELSE 0 END AS iXS_RUC_1T,
    CASE WHEN iPlafondMinimo = 1 THEN 1 ELSE 0 END AS iXS_RUC_PlafondMinimo,
    CASE WHEN iRUC_PA = 1 AND Melhor_Match_RUC IS NOT NULL
              AND Ordem = 2 AND AtividadeCredito = 'Ativo'
         THEN 1 ELSE 0 END AS iXS_RUC_2T,
    -- PCD INATIVOS 1T e 2T
    CASE WHEN iPCD_PA_PM = 1 AND Melhor_Match_PCD IS NOT NULL
              AND Ordem = 1 AND AtividadeCredito = 'Inativo'
         THEN 1 ELSE 0 END AS iXS_PCD_Inativos_1T,
    CASE WHEN iPCD_PA_PM = 1 AND Melhor_Match_PCD IS NOT NULL
              AND Ordem = 2 AND AtividadeCredito = 'Inativo'
         THEN 1 ELSE 0 END AS iXS_PCD_Inativos_2T,
    -- RUC_NPA 1T e 2T
    CASE WHEN iRUC_NPA = 1 AND ((iPCD_PA_PM = 0 AND iPCD_PA_GM = 0) OR Melhor_Match_PCD IS NULL)
              AND (iRUC_PA = 0 OR Melhor_Match_RUC IS NULL)
              AND SubcanalNegocioAtual NOT LIKE 'SPF'
              AND AtividadeCredito <> 'Terminado'
              AND iPlafondMinimo = 0 AND Ordem = 1
         THEN 1 ELSE 0 END AS iXS_RUC_NPA_1T,
    CASE WHEN iRUC_NPA = 1 AND ((iPCD_PA_PM = 0 AND iPCD_PA_GM = 0) OR Melhor_Match_PCD IS NULL)
              AND (iRUC_PA = 0 OR Melhor_Match_RUC IS NULL)
              AND SubcanalNegocioAtual NOT LIKE 'SPF'
              AND AtividadeCredito <> 'Terminado'
              AND iPlafondMinimo = 0 AND Ordem = 2
         THEN 1 ELSE 0 END AS iXS_RUC_NPA_2T,
    -- PCD_NPA 1T e 2T
    CASE WHEN iPCD_NPA = 1 AND ((iPCD_PA_PM = 0 AND iPCD_PA_GM = 0) OR Melhor_Match_PCD IS NULL)
              AND (iRUC_PA = 0 OR Melhor_Match_RUC IS NULL)
              AND SubcanalNegocioAtual NOT LIKE 'SPF'
              AND AtividadeCredito <> 'Terminado'
              AND iPlafondMinimo = 0 AND Ordem = 1
         THEN 1 ELSE 0 END AS iXS_PCD_NPA_1T,
    CASE WHEN iPCD_NPA = 1 AND ((iPCD_PA_PM = 0 AND iPCD_PA_GM = 0) OR Melhor_Match_PCD IS NULL)
              AND (iRUC_PA = 0 OR Melhor_Match_RUC IS NULL)
              AND SubcanalNegocioAtual NOT LIKE 'SPF'
              AND AtividadeCredito <> 'Terminado'
              AND iPlafondMinimo = 0 AND Ordem = 2
         THEN 1 ELSE 0 END AS iXS_PCD_NPA_2T,
    -- TERMINADOS NPA 1T e 2T
    CASE WHEN (iTerminadosPCD = 1 OR iTerminadosRUC = 1)
              AND (MesesTerminoMinimo < 24 and AtividadeCredito LIKE 'Terminado')
              AND Ordem = 1 AND iPlafondMinimo = 0
         THEN 1 ELSE 0 END AS iXS_Terminados_NPA_1T,
    CASE WHEN (iTerminadosPCD = 1 OR iTerminadosRUC = 1)
              AND (MesesTerminoMinimo < 24 and AtividadeCredito LIKE 'Terminado')
              AND Ordem = 2 AND iPlafondMinimo = 0
         THEN 1 ELSE 0 END AS iXS_Terminados_NPA_2T,
    -- TERMINADOS CAPTAÇÃO 1T e 2T
    CASE WHEN (iTerminadosPCD = 1 OR iTerminadosRUC = 1)
              AND Ordem = 1
              AND (MesesTerminoMinimo >= 24 and AtividadeCredito LIKE 'Terminado')
              AND iPlafondMinimo = 0
         THEN 1 ELSE 0 END AS iXS_Terminados_Captacao_1T,
    CASE WHEN (iTerminadosPCD = 1 OR iTerminadosRUC = 1)
              AND Ordem = 2
              AND (MesesTerminoMinimo >= 24 and AtividadeCredito LIKE 'Terminado')
              AND iPlafondMinimo = 0
         THEN 1 ELSE 0 END AS iXS_Terminados_Captacao_2T,
    -- TERMINADOS PA 1T e 2T
    CASE WHEN ((iPCD_PA_PM = 1 OR iPCD_PA_GM = 1 OR iRUC_PA = 1) AND Melhor_Match_PCD IS NOT NULL)
              AND AtividadeCredito = 'Terminado'
              AND MesesTerminoMinimo BETWEEN 0 AND 2
              AND iPlafondMinimo = 0 AND Ordem = 1
         THEN 1 ELSE 0 END AS iXS_Terminados_PA_1T,
    CASE WHEN ((iPCD_PA_PM = 1 OR iPCD_PA_GM = 1 OR iRUC_PA = 1) AND Melhor_Match_PCD IS NOT NULL)
              AND AtividadeCredito = 'Terminado'
              AND MesesTerminoMinimo BETWEEN 0 AND 2
              AND iPlafondMinimo = 0 AND Ordem = 2
         THEN 1 ELSE 0 END AS iXS_Terminados_PA_2T,
    -- RUC/PCD NPA COFIDISPAY
    CASE WHEN iRUC_NPA = 1 AND iPCD_PA_PM = 0 AND iPCD_PA_GM = 0
              AND iRUC_PA = 0 AND SubcanalNegocioAtual LIKE 'CofidisPay'
              AND AtividadeCredito <> 'Terminado' AND iPlafondMinimo = 0
         THEN 1 ELSE 0 END AS iXS_RUC_NPA_CPay,
    CASE WHEN iPCD_NPA = 1 AND iPCD_PA_PM = 0 AND iPCD_PA_GM = 0
              AND SubcanalNegocioAtual LIKE 'CofidisPay'
              AND AtividadeCredito <> 'Terminado' AND iPlafondMinimo = 0
         THEN 1 ELSE 0 END AS iXS_PCD_NPA_CPay,
    -- PMS
    CASE WHEN p.NIF IS NOT NULL THEN 1 ELSE 0 END AS iXS_PMS,
    COALESCE(c.iXS_CCR_SMSA, 0) as iXS_CCR_SMSA,
    COALESCE(c.iXS_CCR_SMSB, 0) as iXS_CCR_SMSB,
    COALESCE(c.iXS_CCR_SMSB, 0) as iXS_CCR_SMSC,
    COALESCE(c.iXS_CCR_SMSB, 0) as iXS_CCR_SMSD,
    COALESCE(c.iXS_CCR, 0) as iXS_CCR,
    iOptOut_Email,
    iOptOut_SMS
INTO #BaseEnvio
FROM #Resultado_Financeiro rf
LEFT JOIN BasePCDPMTOP b ON rf.NIF = b.NIF
LEFT JOIN #ElegiveisPMS p ON p.NIF = rf.NIF
LEFT JOIN (SELECT Montante, NIF, iXS_CCR_SMSA, iXS_CCR_SMSB, iXS_CCR_SMSC, iXS_CCR_SMSD, 1 as iXS_CCR 
           FROM #TabelaFinalCCR) c ON c.NIF = rf.NIF
INNER JOIN armada.dbo.DM_CampanhasElegibilidade_NIF nif ON nif.NIF = rf.NIF;

/* =========================================================
   11. LIMPEZA FINAL
========================================================= */
DELETE FROM #BaseEnvio
WHERE (Email IS NULL AND telemovel IS NULL)  or iQuarentenaFinanciabilidade = 1 
   OR (iXS_PCD_Auto + iXS_PCDPMHabit + iXS_PCDPMTOP + iXS_PCDPM_Parcerias +
       iXS_PCDPM_1T + iXS_PCDPM_2T + iXS_RUC_1T + iXS_RUC_PlafondMinimo +
       iXS_RUC_2T + iXS_PCD_Inativos_1T + iXS_PCD_Inativos_2T + iXS_RUC_NPA_1T +
       iXS_RUC_NPA_2T + iXS_PCD_NPA_1T + iXS_PCD_NPA_2T + iXS_Terminados_NPA_1T +
       iXS_Terminados_NPA_2T + iXS_Terminados_Captacao_1T + iXS_Terminados_Captacao_2T +
       iXS_Terminados_PA_1T + iXS_Terminados_PA_2T + iXS_RUC_NPA_CPay +
       iXS_PCD_NPA_CPay + iXS_PMS + iXS_CCR) = 0;

"""

cursor.execute(query_base_envio)
conn.commit()

df_baseenvio = pd.read_sql("SELECT DISTINCT * FROM #BaseEnvio", conn)

query_resultados = """
DROP TABLE IF EXISTS #Tab0

SELECT DISTINCT 
    cp.NIF, 
    a.IdMes,
    b.IdMes as IdMesAbertura,  
    a.IdProcesso,
    a.DtAtualizacao_PedUnic, 
    a.IdDia,
    b.IdDia as IdDiaAbertura,
    b.MntFinAAbert AS MntAbertura,
    1 AS iPedido,  -- Identifica que esse registro é um pedido
    CASE 
        WHEN b.IdProcesso IS NOT NULL THEN 1  -- Se existe uma correspondência na tabela b (abertura), coloca 1
        ELSE 0  -- Caso contrário, coloca 0
    END AS iAbertura,  -- Identifica se esse registro gerou uma abertura
    a.IdCampanha as IdCampanhaMarcada, -- Adiciona a coluna IdCampanha,
    cd.Descricao,
    cd.Tipo as TipoCampanhaMarcada,
    cd.DataInicio as DataInicioCampanhaMarcada,
    cd.idProduto as IdProdutoCampanhaMarcada,
                 CASE WHEN cd.Descricao LIKE '%R+1%' THEN 'iXS_RUC_R+1'
              WHEN cd.Descricao LIKE '%TOP%' AND cd.IdProduto = 25 THEN 'iXS_PCDPMTOP'
        WHEN cd.Descricao LIKE '%INA%' AND cd.IdProduto = 25 THEN 'iXS_PCD_Inativos_1T'
        WHEN cd.Descricao LIKE '%LAR%' AND cd.IdProduto = 25 THEN 'iXS_PCDPMHabit'
        WHEN cd.Descricao LIKE '%PARCERIAS%' AND cd.IdProduto = 25 THEN 'iXS_PCDPM_Parcerias'
        WHEN (cd.Descricao LIKE '%Auto%' or cd.Descricao LIKE '%MOTO%') and cd.IdProduto = 25 THEN 'iXS_PCD_Auto'
        WHEN cd.Descricao LIKE '%PLAFONDM%' AND cd.IdProduto = 23 THEN 'iXS_RUC_PlafondMinimo'

        WHEN cd.Descricao Like 'CSPCDPA_APT_DIG_JAN23' or cd.Descricao Like 'CSPCDPA_APT_DIG_DEZ22' or cd.Descricao LIKE 'CSPCDPA_APT_DIG' or (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%1TIT%' or cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%' or cd.Descricao LIKE 'CSPCDPA_BYSIDE') AND cd.IdProduto = 25 AND cd.Tipo = 4615) THEN 'iXS_PCDPM_1T'
        WHEN cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%2TIT%' or cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%') AND cd.IdProduto = 25 AND cd.Tipo = 4615 THEN 'iXS_PCDPM_2T'

        WHEN (cd.Descricao LIKE 'CSRUCAUTO_CV') or cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%1TIT%' or cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.IdProduto = 23 AND cd.Tipo = 4615 THEN 'iXS_RUC_1T'
        WHEN cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%2TIT%' or cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%') AND cd.IdProduto = 23 AND cd.Tipo = 4615 THEN 'iXS_RUC_2T'

        WHEN (cd.Descricao LIKE '%CPAY%' AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 25) or cd.Descricao LIKE '%COMBO_CPAY%' or cd.Descricao LIKE 'CSPCD_PAY' or cd.Descricao LIKE 'CSPCD_CPAY'  or cd.Descricao LIKE '%CSPCDCPAY_VOUCHER%'  THEN 'iXS_PCD_NPA_CPay'
        WHEN (cd.Descricao LIKE '%CPAY%' AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 23) or cd.Descricao LIKE '%RUCCPAY%' or cd.Descricao LIKE 'CSRUC_CPAY' THEN 'iXS_RUC_NPA_CPay'

        WHEN (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 25) or cd.Descricao LIKE 'CSPCDSITE_NPA' or cd.Descricao LIKE 'CSPCD_NPA' THEN 'iXS_PCD_NPA_1T'
        WHEN cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%') AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 25 THEN 'iXS_PCD_NPA_2T'
        WHEN (cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 23) or cd.Descricao LIKE 'CSRUCSITE_NPA' or cd.Descricao LIKE 'CSRUC_NPA' THEN 'iXS_RUC_NPA_1T'
        WHEN cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.IdProduto = 23 THEN 'iXS_RUC_NPA_2T'

        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%PA%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_PA_1T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.Descricao LIKE '%PA%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_PA_2T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%CAPT%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_Captacao_1T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.Descricao LIKE '%CAPT%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_Captacao_2T'
        WHEN (cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.IdProduto = 25) or cd.Descricao LIKE 'CSTERM' THEN 'iXS_Terminados_NPA_1T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.IdProduto = 25 THEN 'iXS_Terminados_NPA_2T'

        WHEN cd.IdProduto = 28 THEN 'iXS_PMS'
        WHEN (cd.Descricao LIKE '%SEG_A%' AND cd.IdProduto = 22) or cd.Descricao LIKE 'CSCCR' or cd.Descricao LIKE '%CCR_SMSA%' or cd.Descricao LIKE 'CSCCR_A' or cd.Descricao LIKE '%CCR_SMSA_C2C%' or cd.Descricao LIKE 'CCR_Geral_AtivCofidis' or cd.Descricao LIKE 'CSCCR_LOJ' THEN 'iXS_CCR_SMSA'
        WHEN (cd.Descricao LIKE '%SEG_B%' or cd.Descricao LIKE '%CCR_SMSB%' or cd.Descricao LIKE 'CSCCR_B') AND cd.IdProduto = 22 THEN 'iXS_CCR_SMSB'
        WHEN (cd.Descricao LIKE '%SEG_C%' or cd.Descricao LIKE '%CCR_SMSC%' or cd.Descricao LIKE 'CSCCR_C') AND cd.IdProduto = 22 THEN 'iXS_CCR_SMSC'
        WHEN (cd.Descricao LIKE '%SEG_D%' or cd.Descricao LIKE '%CCR_SMSD%' or cd.Descricao LIKE 'CSCCR_D') AND cd.IdProduto = 22 THEN 'iXS_CCR_SMSD'
        WHEN cd.Descricao LIKE '%T3%' AND cd.IdProduto = 25 THEN 'iXS_RUC_NPA_3T'
        WHEN (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%SITE%') AND cd.IdProduto = 25 AND cd.Tipo = 4615) or (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%SUD%') AND cd.IdProduto = 25 AND cd.Tipo = 4615) THEN 'iXS_PCDPM_1T'
        WHEN (cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%SITE%') AND cd.IdProduto = 23 AND cd.Tipo = 4615) or cd.Descricao LIKE '%CSRUC_SITE%' or (cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%SUD%') AND cd.IdProduto = 23 AND cd.Tipo = 4615) or cd.Descricao LIKE '%CSRUCPA%' THEN 'iXS_RUC_1T'
        ELSE NULL
        END AS iTipoCampanhaMarcada,
                        DATEDIFF(
            DAY,
            CONVERT(date,
                CASE
                    WHEN cd.Descricao LIKE '%R+1%' THEN DataInicio  -- Para RUC_R+1, usa DtCarregamento
                    ELSE NULL  -- Para outras campanhas, usa DataInicio
                END),
            CONVERT(date, a.DtAtualizacao_PedUnic)
        ) AS DifDiasRUCR1,
                CASE
            WHEN DATEDIFF(MONTH,
                          CASE
                              WHEN cd.Descricao LIKE '%R+1%'THEN DataInicio
                              ELSE NULL
                          END,
                          CONVERT(date, a.DtAtualizacao_PedUnic)) < 4
                 THEN 1  -- Efeito RUC
            ELSE 0  -- Não é efeito RUC
        END AS EfeitoRUCR1

        -- Nova coluna iTipoCampanhaMarcado com DataInicio da campanha marcada
INTO #tab0
FROM 
    armada.dbo.DM_ObjetivosDirecaoDetalhe a
INNER JOIN 
    armada.dbo.DM_ClienteProcesso cp 
    ON cp.IdProcesso = a.IdProcesso
LEFT JOIN armada.dbo.DM_CampanhasDefinicao cd on cd.idCampanha = a.IdCampanha 
LEFT JOIN
    armada.dbo.DM_ObjetivosDirecaoDetalhe b
    ON b.IdProcesso = a.IdProcesso
    AND b.IdMes >= 202401
    AND b.canal = 'Directo'
    AND b.subcanal <> 'PRD'
    AND (b.iEstornoMesDif = 0 OR b.iEstornoMesDif IS NULL)
    AND b.TipoFid <> '' 
    AND a.IdCampanha > 0
    AND b.QtdAbert = 1
    AND b.Fonte IN ('Abertura')
WHERE 
    a.IdMes >= 202401 
    AND a.canal = 'Directo' 
    AND a.subcanal <> 'PRD' 
    AND (a.iEstornoMesDif = 0 OR a.iEstornoMesDif IS NULL)
    AND a.TipoFid <> '' 
    AND a.QtdPedidosUnicos = 1
    AND a.IdCampanha > 0
    AND a.Fonte IN ('Pedidos')
    AND cp.NIF <> 0

DROP TABLE IF EXISTS #UltimaCampanhaNIF;

WITH Nivel1 AS (
SELECT a.IdProcesso, a.NIF as NumContribuinte, b.*, CASE WHEN b.IdProcessoA IS NULL THEN 1 ELSE 0 end as iNuncaRecebeuCampanha
FROM #tab0 a
LEFT JOIN (
        SELECT
        p.IdProcesso as IdProcessoA,
        p.NIF as NumContribuinteA,
        ec.IdCampanha,
        ec.MontanteProposto,
        cd.idProduto,
        cd.Texto,
        cd.Tipo,
        cd.Descricao,
        ec.DtCarregamento as DtCarregamentoCampanhaRecebida,
        ec.DtCarregamento,
        cd.DataInicio as DataInicioCampanhadaRecebida, 
        cd.DataInicio, 
        CONVERT(int, CONVERT(VARCHAR(8), ec.DtCarregamento, 112)) AS IdDiaCarreg,
        p.IdDia AS IdDiaProc,

        -- Calculando a diferença de dias, mas sem usar diretamente no ROW_NUMBER
        DATEDIFF(
            DAY,
            CONVERT(date,
                CASE
                    WHEN cd.Descricao LIKE '%R+1%' THEN ec.DtCarregamento  -- Para RUC_R+1, usa DtCarregamento
                    ELSE cd.DataInicio  -- Para outras campanhas, usa DataInicio
                END),
            CONVERT(date, p.DtAtualizacao_PedUnic)
        ) AS DifDias,
        -- Determinar efeito lar (1 ou 0) antes de calcular ROW_NUMBER
        CASE
            WHEN (EfeitoRUCR1 = 0 and DATEDIFF(MONTH,
                          CASE
                              WHEN cd.Descricao LIKE '%R+1%' THEN ec.DtCarregamento
                              ELSE cd.DataInicio
                          END,
                          p.DtAtualizacao_PedUnic) > 4)
                OR ec.DtCarregamento IS NULL THEN 1 
            ELSE 0  -- Não é efeito lar
        END AS EfeitoLar1
    FROM #tab0 p
    LEFT JOIN armada.dbo.DM_EnviosCampanhas ec
        ON p.NIF = ec.NumContribuinte and  p.IdDia >= CONVERT(VARCHAR(8), ec.DtCarregamento, 112)
    LEFT JOIN armada.dbo.DM_CampanhasDefinicao cd
        ON cd.IdCampanha = ec.IdCampanha and cd.TIpo IN (528, 4615)
    WHERE Texto IS NOT NULL and cd.Descricao NOT LIKE '%ELEG%') as b 
on b.idProcessoA = a.IdProcesso and b.IdCampanha IS NOT NULL

),
Nivel2 AS (
    SELECT *,
        ROW_NUMBER() OVER (
            PARTITION BY IdProcesso  -- Agrupando por IdProcesso (independente do NIF)
            ORDER BY 
            CASE WHEN DifDias IS NULL THEN 1 ELSE 0 END, -- Dá prioridade para registros onde DifDias NÃO é nulo
        DifDias ASC   -- Ordenar pela diferença de dias para pegar a campanha mais próxima
        ) AS rn2
    FROM Nivel1
),
CampanhasFinal AS (
    SELECT
        cd.*, CASE WHEN EfeitoLar1 = 1 or iNuncaRecebeuCampanha = 1 THEN 1 ELSE 0 END as EfeitoLar,
        a.IdCampanhaMarcada,
        a.MntAbertura,
        a.iAbertura,
        a.IdMesAbertura, 
        a.IdMes as IdMesPedido,
        a.IdDiaAbertura,
        a.IdDia as IdDiaPedido,
        iTipoCampanhaMarcada,
        EfeitoRUCR1,
        DifDiasRUCR1,

        -- Lógica para definir o tipo de campanha (iTipoCampanha)
          CASE WHEN cd.Descricao LIKE '%R+1%' THEN 'iXS_RUC_R+1'
              WHEN cd.Descricao LIKE '%TOP%' AND cd.IdProduto = 25 THEN 'iXS_PCDPMTOP'
        WHEN cd.Descricao LIKE '%INA%' AND cd.IdProduto = 25 THEN 'iXS_PCD_Inativos_1T'
        WHEN cd.Descricao LIKE '%LAR%' AND cd.IdProduto = 25 THEN 'iXS_PCDPMHabit'
        WHEN cd.Descricao LIKE '%PARCERIAS%' AND cd.IdProduto = 25 THEN 'iXS_PCDPM_Parcerias'
        WHEN (cd.Descricao LIKE '%Auto%' or cd.Descricao LIKE '%MOTO%') and cd.IdProduto = 25 THEN 'iXS_PCD_Auto'
        WHEN cd.Descricao LIKE '%PLAFONDM%' AND cd.IdProduto = 23 THEN 'iXS_RUC_PlafondMinimo'

        WHEN cd.Descricao Like 'CSPCDPA_APT_DIG_JAN23' or cd.Descricao Like 'CSPCDPA_APT_DIG_DEZ22' or cd.Descricao LIKE 'CSPCDPA_APT_DIG' or (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%1TIT%' or cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%' or cd.Descricao LIKE 'CSPCDPA_BYSIDE') AND cd.IdProduto = 25 AND cd.Tipo = 4615) THEN 'iXS_PCDPM_1T'
        WHEN cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%2TIT%' or cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%') AND cd.IdProduto = 25 AND cd.Tipo = 4615 THEN 'iXS_PCDPM_2T'

        WHEN (cd.Descricao LIKE 'CSRUCAUTO_CV') or cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%1TIT%' or cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.IdProduto = 23 AND cd.Tipo = 4615 THEN 'iXS_RUC_1T'
        WHEN cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%2TIT%' or cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%') AND cd.IdProduto = 23 AND cd.Tipo = 4615 THEN 'iXS_RUC_2T'

        WHEN (cd.Descricao LIKE '%CPAY%' AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 25) or cd.Descricao LIKE '%COMBO_CPAY%' or cd.Descricao LIKE 'CSPCD_PAY' or cd.Descricao LIKE 'CSPCD_CPAY'  or cd.Descricao LIKE '%CSPCDCPAY_VOUCHER%'  THEN 'iXS_PCD_NPA_CPay'
        WHEN (cd.Descricao LIKE '%CPAY%' AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 23) or cd.Descricao LIKE '%RUCCPAY%' or cd.Descricao LIKE 'CSRUC_CPAY' THEN 'iXS_RUC_NPA_CPay'

        WHEN (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 25) or cd.Descricao LIKE 'CSPCDSITE_NPA' or cd.Descricao LIKE 'CSPCD_NPA' THEN 'iXS_PCD_NPA_1T'
        WHEN cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%') AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 25 THEN 'iXS_PCD_NPA_2T'
        WHEN (cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 23) or cd.Descricao LIKE 'CSRUCSITE_NPA' or cd.Descricao LIKE 'CSRUC_NPA' THEN 'iXS_RUC_NPA_1T'
        WHEN cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.IdProduto = 23 THEN 'iXS_RUC_NPA_2T'

        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%PA%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_PA_1T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.Descricao LIKE '%PA%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_PA_2T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%CAPT%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_Captacao_1T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.Descricao LIKE '%CAPT%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_Captacao_2T'
        WHEN (cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.IdProduto = 25) or cd.Descricao LIKE 'CSTERM' THEN 'iXS_Terminados_NPA_1T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.IdProduto = 25 THEN 'iXS_Terminados_NPA_2T'

        WHEN cd.IdProduto = 28 THEN 'iXS_PMS'
        WHEN (cd.Descricao LIKE '%SEG_A%' AND cd.IdProduto = 22) or cd.Descricao LIKE 'CSCCR' or cd.Descricao LIKE '%CCR_SMSA%' or cd.Descricao LIKE 'CSCCR_A' or cd.Descricao LIKE '%CCR_SMSA_C2C%' or cd.Descricao LIKE 'CCR_Geral_AtivCofidis' or cd.Descricao LIKE 'CSCCR_LOJ' THEN 'iXS_CCR_SMSA'
        WHEN (cd.Descricao LIKE '%SEG_B%' or cd.Descricao LIKE '%CCR_SMSB%' or cd.Descricao LIKE 'CSCCR_B') AND cd.IdProduto = 22 THEN 'iXS_CCR_SMSB'
        WHEN (cd.Descricao LIKE '%SEG_C%' or cd.Descricao LIKE '%CCR_SMSC%' or cd.Descricao LIKE 'CSCCR_C') AND cd.IdProduto = 22 THEN 'iXS_CCR_SMSC'
        WHEN (cd.Descricao LIKE '%SEG_D%' or cd.Descricao LIKE '%CCR_SMSD%' or cd.Descricao LIKE 'CSCCR_D') AND cd.IdProduto = 22 THEN 'iXS_CCR_SMSD'
        WHEN cd.Descricao LIKE '%T3%' AND cd.IdProduto = 25 THEN 'iXS_RUC_NPA_3T'
        WHEN (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%SITE%') AND cd.IdProduto = 25 AND cd.Tipo = 4615) or (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%SUD%') AND cd.IdProduto = 25 AND cd.Tipo = 4615) THEN 'iXS_PCDPM_1T'
        WHEN (cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%SITE%') AND cd.IdProduto = 23 AND cd.Tipo = 4615) or cd.Descricao LIKE '%CSRUC_SITE%' or (cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%SUD%') AND cd.IdProduto = 23 AND cd.Tipo = 4615) or cd.Descricao LIKE '%CSRUCPA%' THEN 'iXS_RUC_1T'
        ELSE NULL
        END as iTipoCampanha
            -- Adicione mais condições conforme necessário
    FROM Nivel2 cd
    LEFT JOIN #tab0 a ON a.IdProcesso = cd.IdProcesso
    WHERE rn2 = 1
)

SELECT IdProcesso, NumContribuinte, IdDiaProc, IdDiaPedido, 
IdCampanha as IdCampanhaUltimaCampanha,
Descricao as DescricaoCampanhaRecebida, iTipoCampanha as iTipoCampanhaRecebida, 
DtCarregamentoCampanhaRecebida, DataInicio as DataInicioCampanhaRecebida, DifDias, EfeitoLar, DifDiasRUCR1, EfeitoRUCR1, 
IdCampanhaMarcada, iTipoCampanhaMarcada, MntAbertura, iAbertura, IdDiaAbertura,
       -- Definindo o valor de iTipoCampanhaFinal
       CASE
            WHEN EfeitoLar = 1 or EfeitoRUCR1 = 1 THEN  -- Se for efeito lar
                iTipoCampanhaMarcada  -- Usar o tipo da campanha marcada
            ELSE
                iTipoCampanha  -- Caso contrário, usar o tipo da última campanha
        END AS iTipoCampanhaFinal,
            CASE
            WHEN EfeitoLar = 1 or EfeitoRUCR1 = 1 THEN  -- Se for efeito lar
                IdCampanhaMarcada  -- Usar o tipo da campanha marcada
            ELSE
                IdCampanha  -- Caso contrário, usar o tipo da última campanha
        END AS IdCampanhaFinal
INTO #UltimaCampanhaNIF
FROM CampanhasFinal;

DROP TABLE IF EXISTS #PedidosAberturas
SELECT DISTINCT IdProcesso, IdCampanhaUltimaCampanha, IdCampanhaMarcada, DifDias, EfeitoLar, EfeitoRUCR1, DataInicioCampanhaFinal, IdDiaPedido, iAbertura, IdDiaAbertura, MntAbertura, IdCampanhaFinal, iTipoCampanhaFinal
INTO #PedidosAberturas
FROM #UltimaCampanhaNIF n 
LEFT JOIN (SELECT IdCampanha, DataInicio as DataInicioCampanhaFinal  FROM armada.dbo.DM_CampanhasDefinicao cd) as r on r.IdCampanha = n.IdCampanhaFinal
WHERE iTipoCampanhaFinal IS NOT NULL


DROP TABLE IF EXISTS #Envios;
SELECT 
    ec.IdCampanha, 
                  CASE WHEN cd.Descricao LIKE '%R+1%' THEN 'iXS_RUC_R+1'
              WHEN cd.Descricao LIKE '%TOP%' AND cd.IdProduto = 25 THEN 'iXS_PCDPMTOP'
        WHEN cd.Descricao LIKE '%INA%' AND cd.IdProduto = 25 THEN 'iXS_PCD_Inativos_1T'
        WHEN cd.Descricao LIKE '%LAR%' AND cd.IdProduto = 25 THEN 'iXS_PCDPMHabit'
        WHEN cd.Descricao LIKE '%PARCERIAS%' AND cd.IdProduto = 25 THEN 'iXS_PCDPM_Parcerias'
        WHEN (cd.Descricao LIKE '%Auto%' or cd.Descricao LIKE '%MOTO%') and cd.IdProduto = 25 THEN 'iXS_PCD_Auto'
        WHEN cd.Descricao LIKE '%PLAFONDM%' AND cd.IdProduto = 23 THEN 'iXS_RUC_PlafondMinimo'

        WHEN cd.Descricao Like 'CSPCDPA_APT_DIG_JAN23' or cd.Descricao Like 'CSPCDPA_APT_DIG_DEZ22' or cd.Descricao LIKE 'CSPCDPA_APT_DIG' or (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%1TIT%' or cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%' or cd.Descricao LIKE 'CSPCDPA_BYSIDE') AND cd.IdProduto = 25 AND cd.Tipo = 4615) THEN 'iXS_PCDPM_1T'
        WHEN cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%2TIT%' or cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%') AND cd.IdProduto = 25 AND cd.Tipo = 4615 THEN 'iXS_PCDPM_2T'

        WHEN (cd.Descricao LIKE 'CSRUCAUTO_CV') or cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%1TIT%' or cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.IdProduto = 23 AND cd.Tipo = 4615 THEN 'iXS_RUC_1T'
        WHEN cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%2TIT%' or cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%') AND cd.IdProduto = 23 AND cd.Tipo = 4615 THEN 'iXS_RUC_2T'

        WHEN (cd.Descricao LIKE '%CPAY%' AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 25) or cd.Descricao LIKE '%COMBO_CPAY%' or cd.Descricao LIKE 'CSPCD_PAY' or cd.Descricao LIKE 'CSPCD_CPAY'  or cd.Descricao LIKE '%CSPCDCPAY_VOUCHER%'  THEN 'iXS_PCD_NPA_CPay'
        WHEN (cd.Descricao LIKE '%CPAY%' AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 23) or cd.Descricao LIKE '%RUCCPAY%' or cd.Descricao LIKE 'CSRUC_CPAY' THEN 'iXS_RUC_NPA_CPay'

        WHEN (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 25) or cd.Descricao LIKE 'CSPCDSITE_NPA' or cd.Descricao LIKE 'CSPCD_NPA' THEN 'iXS_PCD_NPA_1T'
        WHEN cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%') AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 25 THEN 'iXS_PCD_NPA_2T'
        WHEN (cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 23) or cd.Descricao LIKE 'CSRUCSITE_NPA' or cd.Descricao LIKE 'CSRUC_NPA' THEN 'iXS_RUC_NPA_1T'
        WHEN cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.IdProduto = 23 THEN 'iXS_RUC_NPA_2T'

        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%PA%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_PA_1T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.Descricao LIKE '%PA%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_PA_2T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%CAPT%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_Captacao_1T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.Descricao LIKE '%CAPT%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_Captacao_2T'
        WHEN (cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.IdProduto = 25) or cd.Descricao LIKE 'CSTERM' THEN 'iXS_Terminados_NPA_1T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.IdProduto = 25 THEN 'iXS_Terminados_NPA_2T'

        WHEN cd.IdProduto = 28 THEN 'iXS_PMS'
        WHEN (cd.Descricao LIKE '%SEG_A%' AND cd.IdProduto = 22) or cd.Descricao LIKE 'CSCCR' or cd.Descricao LIKE '%CCR_SMSA%' or cd.Descricao LIKE 'CSCCR_A' or cd.Descricao LIKE '%CCR_SMSA_C2C%' or cd.Descricao LIKE 'CCR_Geral_AtivCofidis' or cd.Descricao LIKE 'CSCCR_LOJ' THEN 'iXS_CCR_SMSA'
        WHEN (cd.Descricao LIKE '%SEG_B%' or cd.Descricao LIKE '%CCR_SMSB%' or cd.Descricao LIKE 'CSCCR_B') AND cd.IdProduto = 22 THEN 'iXS_CCR_SMSB'
        WHEN (cd.Descricao LIKE '%SEG_C%' or cd.Descricao LIKE '%CCR_SMSC%' or cd.Descricao LIKE 'CSCCR_C') AND cd.IdProduto = 22 THEN 'iXS_CCR_SMSC'
        WHEN (cd.Descricao LIKE '%SEG_D%' or cd.Descricao LIKE '%CCR_SMSD%' or cd.Descricao LIKE 'CSCCR_D') AND cd.IdProduto = 22 THEN 'iXS_CCR_SMSD'
        WHEN cd.Descricao LIKE '%T3%' AND cd.IdProduto = 25 THEN 'iXS_RUC_NPA_3T'
        WHEN (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%SITE%') AND cd.IdProduto = 25 AND cd.Tipo = 4615) or (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%SUD%') AND cd.IdProduto = 25 AND cd.Tipo = 4615) THEN 'iXS_PCDPM_1T'
        WHEN (cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%SITE%') AND cd.IdProduto = 23 AND cd.Tipo = 4615) or cd.Descricao LIKE '%CSRUC_SITE%' or (cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%SUD%') AND cd.IdProduto = 23 AND cd.Tipo = 4615) or cd.Descricao LIKE '%CSRUCPA%' THEN 'iXS_RUC_1T'
        ELSE NULL end as iTipoCampanha,
    COUNT(DISTINCT NumContribuinte) AS Envios,
    CONVERT(int, CONVERT(VARCHAR(8), cd.DataInicio, 112)) AS DataInicio
INTO #Envios
FROM armada.dbo.DM_EnviosCampanhas ec
LEFT JOIN armada.dbo.DM_CampanhasDefinicao cd 
    ON cd.idCampanha = ec.IdCampanha
WHERE cd.TIpo IN (528, 4615)
GROUP BY ec.IdCampanha, CONVERT(int, CONVERT(VARCHAR(8), cd.DataInicio, 112)),             CASE WHEN cd.Descricao LIKE '%R+1%' THEN 'iXS_RUC_R+1'
              WHEN cd.Descricao LIKE '%TOP%' AND cd.IdProduto = 25 THEN 'iXS_PCDPMTOP'
        WHEN cd.Descricao LIKE '%INA%' AND cd.IdProduto = 25 THEN 'iXS_PCD_Inativos_1T'
        WHEN cd.Descricao LIKE '%LAR%' AND cd.IdProduto = 25 THEN 'iXS_PCDPMHabit'
        WHEN cd.Descricao LIKE '%PARCERIAS%' AND cd.IdProduto = 25 THEN 'iXS_PCDPM_Parcerias'
        WHEN (cd.Descricao LIKE '%Auto%' or cd.Descricao LIKE '%MOTO%') and cd.IdProduto = 25 THEN 'iXS_PCD_Auto'
        WHEN cd.Descricao LIKE '%PLAFONDM%' AND cd.IdProduto = 23 THEN 'iXS_RUC_PlafondMinimo'

        WHEN cd.Descricao Like 'CSPCDPA_APT_DIG_JAN23' or cd.Descricao Like 'CSPCDPA_APT_DIG_DEZ22' or cd.Descricao LIKE 'CSPCDPA_APT_DIG' or (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%1TIT%' or cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%' or cd.Descricao LIKE 'CSPCDPA_BYSIDE') AND cd.IdProduto = 25 AND cd.Tipo = 4615) THEN 'iXS_PCDPM_1T'
        WHEN cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%2TIT%' or cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%') AND cd.IdProduto = 25 AND cd.Tipo = 4615 THEN 'iXS_PCDPM_2T'

        WHEN (cd.Descricao LIKE 'CSRUCAUTO_CV') or cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%1TIT%' or cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.IdProduto = 23 AND cd.Tipo = 4615 THEN 'iXS_RUC_1T'
        WHEN cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%2TIT%' or cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%') AND cd.IdProduto = 23 AND cd.Tipo = 4615 THEN 'iXS_RUC_2T'

        WHEN (cd.Descricao LIKE '%CPAY%' AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 25) or cd.Descricao LIKE '%COMBO_CPAY%' or cd.Descricao LIKE 'CSPCD_PAY' or cd.Descricao LIKE 'CSPCD_CPAY'  or cd.Descricao LIKE '%CSPCDCPAY_VOUCHER%'  THEN 'iXS_PCD_NPA_CPay'
        WHEN (cd.Descricao LIKE '%CPAY%' AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 23) or cd.Descricao LIKE '%RUCCPAY%' or cd.Descricao LIKE 'CSRUC_CPAY' THEN 'iXS_RUC_NPA_CPay'

        WHEN (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 25) or cd.Descricao LIKE 'CSPCDSITE_NPA' or cd.Descricao LIKE 'CSPCD_NPA' THEN 'iXS_PCD_NPA_1T'
        WHEN cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%') AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 25 THEN 'iXS_PCD_NPA_2T'
        WHEN (cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%NPA%' AND cd.IdProduto = 23) or cd.Descricao LIKE 'CSRUCSITE_NPA' or cd.Descricao LIKE 'CSRUC_NPA' THEN 'iXS_RUC_NPA_1T'
        WHEN cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.IdProduto = 23 THEN 'iXS_RUC_NPA_2T'

        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%PA%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_PA_1T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.Descricao LIKE '%PA%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_PA_2T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.Descricao LIKE '%CAPT%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_Captacao_1T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.Descricao LIKE '%CAPT%' AND cd.IdProduto = 25 THEN 'iXS_Terminados_Captacao_2T'
        WHEN (cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%1T%' or cd.Descricao LIKE '%T1%') AND cd.IdProduto = 25) or cd.Descricao LIKE 'CSTERM' THEN 'iXS_Terminados_NPA_1T'
        WHEN cd.Descricao LIKE '%TERM%' AND (cd.Descricao LIKE '%2T%' or cd.Descricao LIKE '%T2%')  AND cd.IdProduto = 25 THEN 'iXS_Terminados_NPA_2T'

        WHEN cd.IdProduto = 28 THEN 'iXS_PMS'
        WHEN (cd.Descricao LIKE '%SEG_A%' AND cd.IdProduto = 22) or cd.Descricao LIKE 'CSCCR' or cd.Descricao LIKE '%CCR_SMSA%' or cd.Descricao LIKE 'CSCCR_A' or cd.Descricao LIKE '%CCR_SMSA_C2C%' or cd.Descricao LIKE 'CCR_Geral_AtivCofidis' or cd.Descricao LIKE 'CSCCR_LOJ' THEN 'iXS_CCR_SMSA'
        WHEN (cd.Descricao LIKE '%SEG_B%' or cd.Descricao LIKE '%CCR_SMSB%' or cd.Descricao LIKE 'CSCCR_B') AND cd.IdProduto = 22 THEN 'iXS_CCR_SMSB'
        WHEN (cd.Descricao LIKE '%SEG_C%' or cd.Descricao LIKE '%CCR_SMSC%' or cd.Descricao LIKE 'CSCCR_C') AND cd.IdProduto = 22 THEN 'iXS_CCR_SMSC'
        WHEN (cd.Descricao LIKE '%SEG_D%' or cd.Descricao LIKE '%CCR_SMSD%' or cd.Descricao LIKE 'CSCCR_D') AND cd.IdProduto = 22 THEN 'iXS_CCR_SMSD'
        WHEN cd.Descricao LIKE '%T3%' AND cd.IdProduto = 25 THEN 'iXS_RUC_NPA_3T'
        WHEN (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%SITE%') AND cd.IdProduto = 25 AND cd.Tipo = 4615) or (cd.Descricao LIKE '%PCD%' AND (cd.Descricao LIKE '%SUD%') AND cd.IdProduto = 25 AND cd.Tipo = 4615) THEN 'iXS_PCDPM_1T'
        WHEN (cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%SITE%') AND cd.IdProduto = 23 AND cd.Tipo = 4615) or cd.Descricao LIKE '%CSRUC_SITE%' or (cd.Descricao LIKE '%RUC%' AND (cd.Descricao LIKE '%SUD%') AND cd.IdProduto = 23 AND cd.Tipo = 4615) or cd.Descricao LIKE '%CSRUCPA%' THEN 'iXS_RUC_1T'
        ELSE NULL
        END;

DROP TABLE IF EXISTS #ResultadosCampanhas;

WITH Base AS (
    SELECT  
        e.IdCampanha,

        -- Converte DataInicio (YYYYMMDD) para DATE
        TRY_CONVERT(date, CONVERT(char(8), e.DataInicio)) AS DataInicio,

        e.iTipoCampanha,
        e.Envios,

        a.IdProcesso,
        a.iAbertura,
        a.MntAbertura,

        -- Converte YYYYMMDD (INT) para DATE
        TRY_CONVERT(date, CONVERT(char(8), a.IdDiaPedido))   AS DataPedido,
        TRY_CONVERT(date, CONVERT(char(8), a.IdDiaAbertura)) AS DataAbertura
    FROM #Envios e
    LEFT JOIN #PedidosAberturas a
        ON e.IdCampanha = a.IdCampanhaFinal
),

Meses AS (
    SELECT
        *,
        -- diferença em meses entre DataInicio e Pedido/Abertura
        DATEDIFF(MONTH, DataInicio, DataPedido)   AS OffsetPedido,
        DATEDIFF(MONTH, DataInicio, DataAbertura) AS OffsetAbertura
    FROM Base
)

SELECT
    IdCampanha,
    DataInicio,
    iTipoCampanha,
    Envios,

    -- N
    COUNT(DISTINCT CASE WHEN OffsetPedido = 0 THEN IdProcesso END) AS QtdPedidos_N,
    SUM(CASE WHEN OffsetAbertura = 0 THEN iAbertura ELSE 0 END)    AS QtdAberturas_N,
    SUM(CASE WHEN OffsetAbertura = 0 THEN MntAbertura ELSE 0 END)  AS Montante_N,

    -- N+1
    COUNT(DISTINCT CASE WHEN OffsetPedido = 1 THEN IdProcesso END) AS QtdPedidos_N1,
    SUM(CASE WHEN OffsetAbertura = 1 THEN iAbertura ELSE 0 END)    AS QtdAberturas_N1,
    SUM(CASE WHEN OffsetAbertura = 1 THEN MntAbertura ELSE 0 END)  AS Montante_N1,

    -- N+2
    COUNT(DISTINCT CASE WHEN OffsetPedido = 2 THEN IdProcesso END) AS QtdPedidos_N2,
    SUM(CASE WHEN OffsetAbertura = 2 THEN iAbertura ELSE 0 END)    AS QtdAberturas_N2,
    SUM(CASE WHEN OffsetAbertura = 2 THEN MntAbertura ELSE 0 END)  AS Montante_N2,

    -- N+3
    COUNT(DISTINCT CASE WHEN OffsetPedido = 3 THEN IdProcesso END) AS QtdPedidos_N3,
    SUM(CASE WHEN OffsetAbertura = 3 THEN iAbertura ELSE 0 END)    AS QtdAberturas_N3,
    SUM(CASE WHEN OffsetAbertura = 3 THEN MntAbertura ELSE 0 END)  AS Montante_N3,

    -- N+4
    COUNT(DISTINCT CASE WHEN OffsetPedido = 4 THEN IdProcesso END) AS QtdPedidos_N4,
    SUM(CASE WHEN OffsetAbertura = 4 THEN iAbertura ELSE 0 END)    AS QtdAberturas_N4,
    SUM(CASE WHEN OffsetAbertura = 4 THEN MntAbertura ELSE 0 END)  AS Montante_N4
INTO #ResultadosCampanhas
FROM Meses
WHERE iTipoCampanha iS NOT NULL
GROUP BY
    IdCampanha, DataInicio, iTipoCampanha, Envios
ORDER BY DataInicio, IdCampanha;
"""

cursor.execute(query_resultados)
conn.commit()


 # Exemplo: Substituir com uso do pyodbc/pandas read_sql
# df_baseenvio = pd.read_sql(query_base_envio, conexao_sql)

# ------------------------ Consulta dos Resultados da Campanha ----------------
# Substitua pela QUERY específica para obter os resultados das campanhas.
df_resultados_campanha  = pd.read_sql("SELECT DISTINCT * FROM #ResultadosCampanhas WHERE iTipoCampanha IS NOT NULL and iTipoCampanha <> 'iXS_RUC_R+1'", conn)
 # Exemplo: Substituir com uso do pyodbc/pandas read_sql
# df_resultados_campanha = pd.read_sql(query_resultados_campanha, conexao_sql)

# ------------------------ Consulta dos Envios -------------------------------
# Substitua pela QUERY específica para obter os envios.

df_envios = pd.read_sql("SELECT DISTINCT * FROM #Envios WHERE iTipoCampanha IS NOT NULL and iTipoCampanha <> 'iXS_RUC_R+1'", conn)  # Exemplo: Substituir com uso do pyodbc/pandas read_sql
# df_envios = pd.read_sql(query_envios, conexao_sql)

# Validação inicial
if df_baseenvio.empty:
    print("ATENÇÃO: A consulta da #BaseEnvio retornou nenhum dado!")
else:
    print(f"✓ BaseEnvio carregada com {df_baseenvio.shape[0]:,} registros.")

if df_resultados_campanha.empty:
    print("⚠ ATENÇÃO: A consulta da #ResultadosCampanha retornou nenhum dado!")
else:
    print(f"✓ ResultadosCampanha carregada com {df_resultados_campanha.shape[0]:,} registros.")

if df_envios.empty:
    print("⚠ ATENÇÃO: A consulta da #Envios retornou nenhum dado!")
else:
    print(f"✓ Envios carregados com {df_envios.shape[0]:,} registros.")

# =============================================================================
# 1.2 Carregar o Plano de Campanhas
# =============================================================================
# Se o plano for obtido por query SQL


df_plano = pd.read_csv(PATH_PLANO, sep=SEPARADOR, encoding=ENCODING, dtype=str)

  # Substituir o placeholder se necessário.
# df_plano = pd.read_sql(query_plano_campanhas, conexao_sql)

if df_plano.empty:
    print("Nenhuma campanha encontrada no plano! Verifique a sua query.")
else:
    df_plano['data'] = pd.to_datetime(df_plano['data'], dayfirst=True, errors='coerce')
    df_plano['volumetria'] = pd.to_numeric(df_plano['volumetria'], errors='coerce')
    print(f"✓ Plano de campanhas carregado com {df_plano.shape[0]:,} campanhas.")

# Separar campanhas passadas, atuais e futuras (com base na data atual)
campanhas_atuais = df_plano[df_plano['data'] == hoje_str]  # Campanhas para exportação hoje
campanhas_futuras = df_plano[df_plano['data'] > hoje_str]  # Campanhas daqui para frente
campanhas_passadas = df_plano[df_plano['data'] < hoje_str]  # Campanhas que já passaram

print(f"✓ Campanhas para exportar hoje: {len(campanhas_atuais)}")
print(f"✓ Campanhas futuras: {len(campanhas_futuras)}")
print(f"✓ Campanhas passadas ignoradas: {len(campanhas_passadas)}")

# =============================================================================
# 1.3 Resumo
# =============================================================================

print("\nResumo da inicialização:")
print(f"  - #BaseEnvio: {df_baseenvio.shape[0]} clientes carregados.")
print(f"  - #ResultadosCampanha: {df_resultados_campanha.shape[0]} registros carregados.")
print(f"  - #Envios: {df_envios.shape[0]} registros carregados.")
print(f"  - Plano de Campanhas: {df_plano.shape[0]:,} campanhas ativas no plano.")


GESTOR DE CAMPANHAS E GESTOR DE ESCASSEZ

1. CONFIGURAÇÃO INICIAL E CONSULTAS AOS DADOS

1. CONFIGURAÇÃO
✓ Data Atual: 2026-02-16
✓ Mês Atual: 202602
✓ Mês Anterior: 202601
✓ Campanha Prioritária: iXS_PCDPMTOP


C:\Users\COTAGO\AppData\Local\Temp\ipykernel_5004\2432963391.py:1066: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_baseenvio = pd.read_sql("SELECT DISTINCT * FROM #BaseEnvio", conn)


✓ BaseEnvio carregada com 373,360 registros.
✓ ResultadosCampanha carregada com 702 registros.
✓ Envios carregados com 702 registros.
✓ Plano de campanhas carregado com 28 campanhas.
✓ Campanhas para exportar hoje: 1
✓ Campanhas futuras: 6
✓ Campanhas passadas ignoradas: 21

Resumo da inicialização:
  - #BaseEnvio: 373360 clientes carregados.
  - #ResultadosCampanha: 702 registros carregados.
  - #Envios: 702 registros carregados.
  - Plano de Campanhas: 28 campanhas ativas no plano.


C:\Users\COTAGO\AppData\Local\Temp\ipykernel_5004\2432963391.py:1498: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resultados_campanha  = pd.read_sql("SELECT DISTINCT * FROM #ResultadosCampanhas WHERE iTipoCampanha IS NOT NULL and iTipoCampanha <> 'iXS_RUC_R+1'", conn)
C:\Users\COTAGO\AppData\Local\Temp\ipykernel_5004\2432963391.py:1505: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_envios = pd.read_sql("SELECT DISTINCT * FROM #Envios WHERE iTipoCampanha IS NOT NULL and iTipoCampanha <> 'iXS_RUC_R+1'", conn)  # Exemplo: Substituir com uso do pyodbc/pandas read_sql


In [59]:
query_resultados = """
DROP TABLE IF EXISTS #Resultados
select IdMes, case WHEN a1.TipoFid NOT LIKE 'FCC' and a1.TipoFid NOT LIKE 'FSC' THEN 'Captação'
        WHEN a1.TipoFid = 'FCC' and x.iTipoCampanhaFinal NOT LIKE 'iXS_RUC_R+1' then 'XS'
        ELSE 'Outras XS' end as TipoMKT, sum(QtdPedidosUnicos) as N_Pedidos, sum(QtdAbert) as N_Aberturas, sum(MntFinAAbert) as Financiamentos
INTO #Resultados
from armada.dbo.DM_ObjetivosDirecaoDetalhe as a1
LEFT JOIN #PedidosAberturas x on x.IdProcesso = a1.IdProcesso
where idmes>=202301 and (iEstornoMesDif=0 or iEstornoMesDif is null) and a1.canal='Directo' and a1.SubCanal<>'PRD' 
and Fonte in ('Abertura','Pedidos')
group by IdMes, case WHEN a1.TipoFid NOT LIKE 'FCC' and a1.TipoFid NOT LIKE 'FSC' THEN 'Captação'
        WHEN a1.TipoFid = 'FCC' and x.iTipoCampanhaFinal NOT LIKE 'iXS_RUC_R+1' then 'XS'
        ELSE 'Outras XS' end 
order by IdMes, case WHEN a1.TipoFid NOT LIKE 'FCC' and a1.TipoFid NOT LIKE 'FSC' THEN 'Captação'
        WHEN a1.TipoFid = 'FCC' and x.iTipoCampanhaFinal NOT LIKE 'iXS_RUC_R+1' then 'XS'
        ELSE 'Outras XS' end 
"""

cursor.execute(query_resultados)
conn.commit()
df_resultados = pd.read_sql("SELECT DISTINCT * FROM #Resultados", conn)  # Exemplo: Substituir com uso do pyodbc/pandas read_sql
# df_envios = pd.read_sql(query_envios, conexao_sql)

C:\Users\COTAGO\AppData\Local\Temp\ipykernel_5004\2129375966.py:21: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_resultados = pd.read_sql("SELECT DISTINCT * FROM #Resultados", conn)  # Exemplo: Substituir com uso do pyodbc/pandas read_sql


In [60]:
query_objetivos = """
DROP TABLE IF EXISTS #Objetivos 
select IdMes, TipoMarketing, SUM(CASE WHEN Objetivo_Tipo LIKE 'Pedidos' THEN Objetivo ELSE NULL end) as Pedidos,
SUM(CASE WHEN Objetivo_Tipo LIKE 'Aberturas' THEN Objetivo ELSE NULL end) as Aberturas,
SUM(CASE WHEN Objetivo_Tipo LIKE 'Finanaciamentos à abertura' THEN Objetivo ELSE NULL end) as Financiamentos 
INTO #Objetivos
from armada.dbo.DM_ObjetivosBEP_DICV 
WHERE Objetivo_Tipo IN ('Finanaciamentos à abertura', 'Pedidos', 'Aberturas') and IdMes >= 202601
GROUP BY IdMes, TipoMarketing
"""

cursor.execute(query_objetivos)
conn.commit()
df_objetivos = pd.read_sql("SELECT DISTINCT * FROM #Objetivos", conn)  # Exemplo: Substituir com uso do pyodbc/pandas read_sql
# df_envios = pd.read_sql(query_envios, conexao_sql)

C:\Users\COTAGO\AppData\Local\Temp\ipykernel_5004\2600768619.py:14: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_objetivos = pd.read_sql("SELECT DISTINCT * FROM #Objetivos", conn)  # Exemplo: Substituir com uso do pyodbc/pandas read_sql


In [61]:
# =============================================================================
# BLOCO 1: ESCASSEZ & OVERLAP (apenas campanhas de hoje até fim do mês)
# =============================================================================
print("\n1. ESCASSEZ & OVERLAP DAS CAMPANHAS FUTURAS (MÊS CORRENTE)\n")

# Assumimos:
# df_baseenvio, df_plano já carregados
# hoje_str já definido (string)
# hoje definido como datetime (se não, criamos)
if 'hoje' not in globals():
    hoje = pd.to_datetime(hoje_str, errors='coerce')

# 1) Normalizar datas no plano
df_plano['data'] = pd.to_datetime(df_plano['data'], dayfirst=True, errors='coerce')

# 2) Filtrar apenas campanhas do mês atual (hoje até fim do mês)
fim_mes = hoje + pd.offsets.MonthEnd(0)
df_plano_mes = df_plano[(df_plano['data'] >= hoje) & (df_plano['data'] <= fim_mes)].copy()

# 3) Identificar colunas de elegibilidade (iXS_)
colunas_elegibilidade = [c for c in df_baseenvio.columns if c.startswith('iXS_')]

# 4) Campanhas do plano (validar se existem na base)
campanhas_plano = df_plano_mes['campanha'].dropna().unique().tolist()
campanhas_validas = [c for c in campanhas_plano if c in colunas_elegibilidade]

if len(campanhas_validas) == 0:
    raise ValueError("⚠️ Nenhuma campanha do #Plano deste mês existe na #BaseEnvio (colunas iXS_...).")

# 5) Calcular escassez por campanha (com base no #Plano filtrado)
indice_escassez = {}
detalhes_escassez = {}

for _, row in df_plano_mes.iterrows():
    campanha = row['campanha']
    if campanha not in campanhas_validas:
        continue

    necessidade = int(row['volumetria']) if pd.notna(row['volumetria']) else 0
    data_exec = row['data']

    elegiveis_totais = int(df_baseenvio[campanha].sum()) if campanha in df_baseenvio.columns else 0
    escassez = (necessidade / elegiveis_totais) if elegiveis_totais > 0 else 1.0

    dias_ate = (data_exec - hoje).days if pd.notna(data_exec) else None

    indice_escassez[campanha] = escassez
    detalhes_escassez[campanha] = {
        'necessidade': necessidade,
        'elegiveis_totais': elegiveis_totais,
        'escassez': escassez,
        'dias_ate': dias_ate,
        'data': data_exec.strftime('%d/%m/%Y') if pd.notna(data_exec) else 'N/A'
    }

# 6) Classificação de criticidade
def classificar_escassez(x):
    if x > 0.8:
        return "CRÍTICO"
    elif x > 0.5:
        return "ALTO"
    elif x > 0.3:
        return "MÉDIO"
    else:
        return "BAIXO"

df_escassez = pd.DataFrame([
    {
        'campanha': c,
        'necessidade': d['necessidade'],
        'elegiveis_totais': d['elegiveis_totais'],
        'escassez': d['escassez'],
        'status': classificar_escassez(d['escassez']),
        'dias_ate': d['dias_ate'],
        'data': d['data']
    }
    for c, d in detalhes_escassez.items()
])

print("✓ Escassez calculada (campanhas futuras do mês):")
print(df_escassez.sort_values('escassez', ascending=False).to_string(index=False))

# 7) Overlap apenas entre campanhas do mês atual
from collections import defaultdict

overlap = defaultdict(dict)

sets_elegiveis = {
    c: set(df_baseenvio.loc[df_baseenvio[c] == 1, 'NIF'])
    for c in campanhas_validas
}

for c1 in campanhas_validas:
    for c2 in campanhas_validas:
        if c1 == c2:
            overlap[c1][c2] = {'overlap_abs': 0, 'overlap_rel': 0}
            continue

        intersecao = sets_elegiveis[c1].intersection(sets_elegiveis[c2])
        overlap_abs = len(intersecao)

        menor = min(len(sets_elegiveis[c1]), len(sets_elegiveis[c2]))
        overlap_rel = (overlap_abs / menor) if menor > 0 else 0

        overlap[c1][c2] = {
            'overlap_abs': overlap_abs,
            'overlap_rel': overlap_rel
        }

print("\n✓ Overlap calculado (campanhas do mês)")

# 8) Marcar campanhas já executadas no plano original
df_plano['status_execucao'] = df_plano['data'].apply(
    lambda d: 'EXECUTADA' if pd.notna(d) and d < hoje else 'PLANEADA'
)

# 9) Guardar escassez/prioridade no df_plano (apenas as do mês)
df_plano['escassez'] = df_plano['campanha'].map(indice_escassez)
df_plano['prioridade'] = df_plano['escassez'].apply(lambda x: classificar_escassez(x) if pd.notna(x) else None)

print("✓ Escassez e prioridade adicionadas ao #Plano (mês corrente).")



1. ESCASSEZ & OVERLAP DAS CAMPANHAS FUTURAS (MÊS CORRENTE)

✓ Escassez calculada (campanhas futuras do mês):
             campanha  necessidade  elegiveis_totais  escassez  status  dias_ate       data
 iXS_Terminados_PA_2T         5000               171 29.239766 CRÍTICO         7 24/02/2026
iXS_RUC_PlafondMinimo         1020               157  6.496815 CRÍTICO         7 24/02/2026
  iXS_PCD_Inativos_2T         4000              3436  1.164144 CRÍTICO         7 24/02/2026
  iXS_PCD_Inativos_1T         4060             18857  0.215305   BAIXO         7 24/02/2026
       iXS_PCDPMHabit         2000             35060  0.057045   BAIXO         1 18/02/2026
         iXS_PCD_Auto         2000            117564  0.017012   BAIXO         1 18/02/2026

✓ Overlap calculado (campanhas do mês)
✓ Escassez e prioridade adicionadas ao #Plano (mês corrente).


In [62]:
query_pace = """
drop table if exists #pace_montante_long;
drop table if exists #pace_montante;
drop table if exists #pace_aberturas_long;
drop table if exists #pace_aberturas;
drop table if exists #pace_pedidos_long;
drop table if exists #pace_pedidos;
drop table if exists #cols;
   
WITH N AS (
    SELECT TOP (101) ROW_NUMBER() OVER (ORDER BY (SELECT NULL)) - 1 AS d
    FROM sys.all_objects
),
Cols AS (
    SELECT STRING_AGG(QUOTENAME('D' + CAST(d AS varchar(3))), ',') AS cols
    FROM N
)
SELECT cols INTO #cols FROM Cols;

/* ============================================================
   2) PEDIDOS (sempre)
   ============================================================ */
WITH base_ped AS (
    SELECT
        iTipoCampanhaFinal,
        DATEDIFF(day,
            CAST(DataInicioCampanhaFinal AS date),
            CAST(CONVERT(date, CONVERT(varchar(8), IdDiaPedido)) AS date)
        ) AS diff_dia
    FROM #PedidosAberturas
    WHERE IdDiaPedido IS NOT NULL
),
ped_agg AS (
    SELECT iTipoCampanhaFinal, diff_dia, COUNT(*) AS qtd
    FROM base_ped
    WHERE diff_dia BETWEEN 0 AND 100
    GROUP BY iTipoCampanhaFinal, diff_dia
),
ped_tot AS (
    SELECT iTipoCampanhaFinal, SUM(qtd) AS total
    FROM ped_agg
    GROUP BY iTipoCampanhaFinal
),
ped_pace AS (
    SELECT
        a.iTipoCampanhaFinal,
        'D' + CAST(a.diff_dia AS varchar(3)) AS dia,
        CAST(a.qtd AS float) / NULLIF(t.total,0) AS pace
    FROM ped_agg a
    JOIN ped_tot t ON t.iTipoCampanhaFinal = a.iTipoCampanhaFinal
)
SELECT *
INTO #pace_pedidos_long
FROM ped_pace;

/* Pivot PEDIDOS */
DECLARE @cols_ped nvarchar(max) = (SELECT cols FROM #cols);
DECLARE @sql_ped nvarchar(max) =
N'SELECT iTipoCampanhaFinal, ' + @cols_ped + '
  INTO #pace_pedidos
  FROM #pace_pedidos_long
  PIVOT (MAX(pace) FOR dia IN (' + @cols_ped + ')) p;';
EXEC sp_executesql @sql_ped;


/* ============================================================
   3) ABERTURAS (ignora NULL)
   ============================================================ */
WITH base_ab AS (
    SELECT
        iTipoCampanhaFinal,
        DATEDIFF(day,
            CAST(DataInicioCampanhaFinal AS date),
            CAST(CONVERT(date, CONVERT(varchar(8), IdDiaAbertura)) AS date)
        ) AS diff_dia
    FROM #PedidosAberturas
    WHERE IdDiaAbertura IS NOT NULL
),
ab_agg AS (
    SELECT iTipoCampanhaFinal, diff_dia, COUNT(*) AS qtd
    FROM base_ab
    WHERE diff_dia BETWEEN 0 AND 100
    GROUP BY iTipoCampanhaFinal, diff_dia
),
ab_tot AS (
    SELECT iTipoCampanhaFinal, SUM(qtd) AS total
    FROM ab_agg
    GROUP BY iTipoCampanhaFinal
),
ab_pace AS (
    SELECT
        a.iTipoCampanhaFinal,
        'D' + CAST(a.diff_dia AS varchar(3)) AS dia,
        CAST(a.qtd AS float) / NULLIF(t.total,0) AS pace
    FROM ab_agg a
    JOIN ab_tot t ON t.iTipoCampanhaFinal = a.iTipoCampanhaFinal
)
SELECT *
INTO #pace_aberturas_long
FROM ab_pace;

/* Pivot ABERTURAS */
DECLARE @cols_ab nvarchar(max) = (SELECT cols FROM #cols);
DECLARE @sql_ab nvarchar(max) =
N'SELECT iTipoCampanhaFinal, ' + @cols_ab + '
  INTO #pace_aberturas
  FROM #pace_aberturas_long
  PIVOT (MAX(pace) FOR dia IN (' + @cols_ab + ')) p;';
EXEC sp_executesql @sql_ab;


/* ============================================================
   4) FINANCIAMENTOS (MntAbertura, ignora NULL)
   ============================================================ */
WITH base_mon AS (
    SELECT
        iTipoCampanhaFinal,
        DATEDIFF(day,
            CAST(DataInicioCampanhaFinal AS date),
            CAST(CONVERT(date, CONVERT(varchar(8), IdDiaAbertura)) AS date)
        ) AS diff_dia,
        MntAbertura
    FROM #PedidosAberturas
    WHERE MntAbertura IS NOT NULL
      AND IdDiaAbertura IS NOT NULL
),
mon_agg AS (
    SELECT iTipoCampanhaFinal, diff_dia, SUM(MntAbertura) AS mnt
    FROM base_mon
    WHERE diff_dia BETWEEN 0 AND 100
    GROUP BY iTipoCampanhaFinal, diff_dia
),
mon_tot AS (
    SELECT iTipoCampanhaFinal, SUM(mnt) AS total
    FROM mon_agg
    GROUP BY iTipoCampanhaFinal
),
mon_pace AS (
    SELECT
        a.iTipoCampanhaFinal,
        'D' + CAST(a.diff_dia AS varchar(3)) AS dia,
        CAST(a.mnt AS float) / NULLIF(t.total,0) AS pace
    FROM mon_agg a
    JOIN mon_tot t ON t.iTipoCampanhaFinal = a.iTipoCampanhaFinal
)
SELECT *
INTO #pace_montante_long
FROM mon_pace;

/* Pivot MONTANTE */
DECLARE @cols_mon nvarchar(max) = (SELECT cols FROM #cols);
DECLARE @sql_mon nvarchar(max) =
N'SELECT iTipoCampanhaFinal, ' + @cols_mon + '
  INTO #pace_montante
  FROM #pace_montante_long
  PIVOT (MAX(pace) FOR dia IN (' + @cols_mon + ')) p;';
EXEC sp_executesql @sql_mon"""



In [63]:
cursor.execute(query_pace)
conn.commit()


df_pace_pedidos_long  = pd.read_sql("SELECT DISTINCT * FROM #pace_pedidos_long WHERE iTipoCampanhaFinal IS NOT NULL and iTipoCampanhaFinal <> 'iXS_RUC_R+1'", conn)
df_pace_aberturas_long  = pd.read_sql("SELECT DISTINCT * FROM #pace_aberturas_long WHERE iTipoCampanhaFinal IS NOT NULL and iTipoCampanhaFinal <> 'iXS_RUC_R+1'", conn)
df_pace_montante_long  = pd.read_sql("SELECT DISTINCT * FROM #pace_montante_long WHERE iTipoCampanhaFinal IS NOT NULL and iTipoCampanhaFinal <> 'iXS_RUC_R+1'", conn)

C:\Users\COTAGO\AppData\Local\Temp\ipykernel_5004\4049523445.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pace_pedidos_long  = pd.read_sql("SELECT DISTINCT * FROM #pace_pedidos_long WHERE iTipoCampanhaFinal IS NOT NULL and iTipoCampanhaFinal <> 'iXS_RUC_R+1'", conn)
C:\Users\COTAGO\AppData\Local\Temp\ipykernel_5004\4049523445.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pace_aberturas_long  = pd.read_sql("SELECT DISTINCT * FROM #pace_aberturas_long WHERE iTipoCampanhaFinal IS NOT NULL and iTipoCampanhaFinal <> 'iXS_RUC_R+1'", conn)
C:\Users\COTAGO\AppData\Local\Temp\ipykernel_5004\4049523445.py:7: UserWarning: pandas only supports SQLAlchemy connectable (eng

In [64]:
df_pace_pedidos = df_pace_pedidos_long.pivot_table(
    index='iTipoCampanhaFinal', columns='dia', values='pace', aggfunc='mean'
).reset_index()

df_pace_pedidos = df_pace_pedidos.rename(columns={'iTipoCampanhaFinal':'iTipoCampanha'})

df_pace_aberturas = df_pace_aberturas_long.pivot_table(
    index='iTipoCampanhaFinal', columns='dia', values='pace', aggfunc='mean'
).reset_index()

df_pace_aberturas = df_pace_aberturas.rename(columns={'iTipoCampanhaFinal':'iTipoCampanha'})

df_pace_montante = df_pace_montante_long.pivot_table(
    index='iTipoCampanhaFinal', columns='dia', values='pace', aggfunc='mean'
).reset_index()

df_pace_montante = df_pace_montante.rename(columns={'iTipoCampanhaFinal':'iTipoCampanha'})

In [ ]:
# =============================================================================
# BLOCO 2 (AJUSTADO) — INTEGRAL (com média 3M + dias úteis PT + taxas 12M ponderadas)
# =============================================================================
print("\n2. CENÁRIOS + RESUMO COMPARATIVO (com metas df_objetivos)\n")

from workalendar.europe import Portugal
from dateutil.relativedelta import relativedelta
import calendar
import sys

cal = Portugal()

def dias_uteis_entre(inicio, fim):
    dias = pd.date_range(start=inicio, end=fim, freq='D')
    return sum(cal.is_working_day(d.date()) for d in dias)

def fim_do_mes(dt):
    return pd.Timestamp(year=dt.year, month=dt.month, day=calendar.monthrange(dt.year, dt.month)[1])

def idmes(dt):
    return int(f"{dt.year}{dt.month:02d}")

def dias_uteis_ate(data_ref):
    inicio = pd.Timestamp(data_ref.year, data_ref.month, 1)
    dias = pd.date_range(start=inicio, end=data_ref, freq='D')
    return sum(cal.is_working_day(d.date()) for d in dias)

def deve_executar(hoje):
    if not cal.is_working_day(hoje.date()):
        return False
    du = dias_uteis_ate(hoje)
    return (du == 1) or ((du - 1) % 5 == 0)

if 'hoje' not in globals():
    hoje = pd.to_datetime(hoje_str, errors='coerce')

if not deve_executar(hoje):
    print("Hoje não é dia de execução.")
    sys.exit(0)

inicio_mes_atual = pd.Timestamp(hoje.year, hoje.month, 1)
fim_mes = fim_do_mes(hoje)

# dias úteis do mês corrente
dias_total = dias_uteis_entre(inicio_mes_atual, fim_mes)

# últimos 12 meses fechados (exclui mês corrente)
meses_fechados = [inicio_mes_atual - relativedelta(months=i) for i in range(1, 13)]
ids_12m = [idmes(d) for d in meses_fechados]

# mapa dias úteis por mês fechado
dias_uteis_map = {idmes(d): dias_uteis_entre(d, fim_do_mes(d)) for d in meses_fechados}

# -------------------------------------------------------------------------
# 2.1 Projeções por tipoMKT (média 3 meses fechados, ajustada por dias úteis)
# -------------------------------------------------------------------------
df_resultados['IdMes'] = df_resultados['IdMes'].astype(int)

ids_3m = ids_12m[:3]  # últimos 3 meses fechados (mais recentes)

def ajustar_mes(valor, id_mes):
    du = dias_uteis_map.get(id_mes, 0)
    return (valor / du) * dias_total if du > 0 else 0

df_3m = df_resultados[df_resultados['IdMes'].isin(ids_3m)].copy()
df_3m['Fin_ajust'] = df_3m.apply(lambda r: ajustar_mes(r['Financiamentos'], r['IdMes']), axis=1)

proj = df_3m.groupby('TipoMKT', as_index=False)['Fin_ajust'].mean()
fin_xs = float(proj.loc[proj['TipoMKT'] == 'XS', 'Fin_ajust'].sum())
fin_outras_xs = float(proj.loc[proj['TipoMKT'] == 'Outras XS', 'Fin_ajust'].sum())
fin_capt = float(proj.loc[proj['TipoMKT'] == 'captação', 'Fin_ajust'].sum())

# -------------------------------------------------------------------------
# 2.2 Metas do mês (df_objetivos)
# -------------------------------------------------------------------------
df_objetivos['IdMes'] = df_objetivos['IdMes'].astype(int)
idmes_atual = idmes(inicio_mes_atual)

obj_mes = df_objetivos[df_objetivos['IdMes'] == idmes_atual].copy()

meta_fid = float(obj_mes.loc[obj_mes['TipoMarketing'] == 'Fidelização', 'Financiamentos'].sum())
meta_capt = float(obj_mes.loc[obj_mes['TipoMarketing'] == 'Captação', 'Financiamentos'].sum())

gap_capt = max(0, meta_capt - fin_capt)

# -------------------------------------------------------------------------
# 2.3 Taxas por campanha (N0..N4) — 12 meses fechados, ponderadas por envios
#      com exclusões: N1 exclui 1 mês, N2 exclui 2, N3 exclui 3, N4 exclui 4
#      e valores ajustados por dias úteis antes da ponderação
# -------------------------------------------------------------------------
df_resultados_campanha['DataInicio'] = pd.to_datetime(
    df_resultados_campanha['DataInicio'], dayfirst=True, errors='coerce'
)
df_resultados_campanha['IdMes'] = df_resultados_campanha['DataInicio'].apply(lambda d: idmes(d) if pd.notna(d) else None)

def calcular_taxa_ponderada(df_camp, offset, col_base):
    excluir = offset
    meses_validos = ids_12m[excluir:]

    df = df_camp[df_camp['IdMes'].isin(meses_validos)].copy()
    if df.empty:
        return 0.0

    col_val = col_base if offset == 0 else f"{col_base}{offset}"

    def val_ajust(row):
        val = float(row.get(col_val, 0) or 0)
        du = dias_uteis_map.get(row['IdMes'], 0)
        return (val / du) * dias_total if du > 0 else 0

    df['Val_ajust'] = df.apply(val_ajust, axis=1)
    total_envios = df['Envios'].sum()
    if total_envios <= 0:
        return 0.0

    return df['Val_ajust'].sum() / total_envios

taxas = []
for campanha, grp in df_resultados_campanha.groupby('iTipoCampanha'):
    for offset in [0,1,2,3,4]:
        taxas.append({
            'campanha': campanha,
            'offset': offset,
            'taxa_montante_mes': calcular_taxa_ponderada(grp, offset, "Montante_N"),
            'taxa_pedidos_mes': calcular_taxa_ponderada(grp, offset, "QtdPedidos_N"),
            'taxa_aberturas_mes': calcular_taxa_ponderada(grp, offset, "QtdAberturas_N"),
        })

taxa_media = pd.DataFrame(taxas)

# -------------------------------------------------------------------------
# 2.4 Campanhas do mês (df_envios e df_plano)
# -------------------------------------------------------------------------
df_envios['DataInicio'] = pd.to_datetime(df_envios['DataInicio'], dayfirst=True, errors='coerce')

def diff_meses(dt_inicio, dt_referencia):
    return (dt_referencia.year - dt_inicio.year) * 12 + (dt_referencia.month - dt_inicio.month)

df_exec = df_envios[df_envios['DataInicio'] < hoje].copy()
df_exec['offset'] = df_exec['DataInicio'].apply(lambda d: diff_meses(d, inicio_mes_atual))
df_exec = df_exec[df_exec['offset'].between(0, 4)]
df_exec = df_exec.rename(columns={'iTipoCampanha':'campanha','Envios':'nifs_alocados'})

df_fut = df_plano[(df_plano['data'] >= hoje) & (df_plano['data'] <= fim_mes)].copy()
df_fut['offset'] = 0

if 'campanha' not in df_fut.columns:
    if 'iTipoCampanha' in df_fut.columns:
        df_fut = df_fut.rename(columns={'iTipoCampanha':'campanha'})
    elif 'IdCampanha' in df_fut.columns:
        df_fut = df_fut.rename(columns={'IdCampanha':'campanha'})

if 'volumetria' not in df_fut.columns and 'Envios' in df_fut.columns:
    df_fut = df_fut.rename(columns={'Envios':'volumetria'})

# -------------------------------------------------------------------------
# 2.5 Base elegíveis + alocação
# -------------------------------------------------------------------------
colunas_elegibilidade = [c for c in df_baseenvio.columns if c.startswith('iXS_')]

clientes_elegiveis = {}
for _, row in df_baseenvio.iterrows():
    nif = row['NIF']
    elegiveis_para = [c for c in colunas_elegibilidade if row.get(c, 0) == 1]
    clientes_elegiveis[nif] = {
        'elegiveis_para': elegiveis_para,
        'num_elegiveis': len(elegiveis_para),
        'exclusivo': len(elegiveis_para) == 1
    }

CAMPANHAS_SEM_BLOQUEIO_MES_ANTERIOR = {'iXS_Terminados_', 'iXS_RUC_PlafondMinimo'}
mes_atual = idmes_atual
mes_anterior = idmes(inicio_mes_atual - relativedelta(months=1))

def verificar_bloqueio(idmes_ult, campanhas_cliente):
    try:
        idmes_ult = int(idmes_ult)
        if idmes_ult == mes_atual:
            return True
        if idmes_ult == mes_anterior:
            return not any(c.startswith(tuple(CAMPANHAS_SEM_BLOQUEIO_MES_ANTERIOR)) for c in campanhas_cliente)
        return False
    except:
        return False

clientes_contactaveis = {}
for nif, info in clientes_elegiveis.items():
    if not info['elegiveis_para']:
        continue
    idmes_ult = df_baseenvio.loc[df_baseenvio['NIF'] == nif, 'IdMesUltimaCampanha'].values[0]
    bloqueado = verificar_bloqueio(idmes_ult, info['elegiveis_para'])
    if not bloqueado:
        clientes_contactaveis[nif] = info

def alocar_por_target(campanhas_df, target_fin, scenario_label):
    plan_weights = campanhas_df.set_index('campanha')['volumetria'].fillna(0)
    eleg_counts = {c:0 for c in campanhas_df['campanha'].unique()}
    for nif, info in clientes_contactaveis.items():
        for c in info['elegiveis_para']:
            if c in eleg_counts:
                eleg_counts[c] += 1
    eleg_weights = pd.Series(eleg_counts)

    if plan_weights.sum() > 0:
        plan_weights = plan_weights / plan_weights.sum()
    if eleg_weights.sum() > 0:
        eleg_weights = eleg_weights / eleg_weights.sum()

    taxa_camp = taxa_media[taxa_media['offset'] == 0].set_index('campanha')['taxa_montante_mes']
    perf_weights = taxa_camp / taxa_camp.sum() if taxa_camp.sum() > 0 else taxa_camp

    weights = 0.5 * plan_weights + 0.3 * eleg_weights + 0.2 * perf_weights
    weights = weights.fillna(0)

    resultados = []
    clientes_utilizados = set()

    for campanha, w in weights.items():
        if campanha not in colunas_elegibilidade:
            continue
        if w == 0:
            continue

        taxa = float(taxa_camp.get(campanha, 0))
        if taxa <= 0:
            continue

        target_camp = target_fin * w
        nifs_necessarios = int(round(target_camp / taxa))

        nifs_disp = [
            nif for nif, info in clientes_contactaveis.items()
            if campanha in info['elegiveis_para'] and nif not in clientes_utilizados
        ]

        nifs_alocar = min(nifs_necessarios, len(nifs_disp))

        for nif in nifs_disp[:nifs_alocar]:
            clientes_utilizados.add(nif)

        resultados.append({
            'campanha': campanha,
            'nifs_alocados': nifs_alocar,
            'volumetria': nifs_necessarios,
            'offset': 0,
            'scenario': scenario_label
        })

    return pd.DataFrame(resultados, columns=['campanha','nifs_alocados','volumetria','offset','scenario'])

def alocar_todos(campanhas_df, scenario_label):
    resultados = []
    for campanha in campanhas_df['campanha'].unique():
        if campanha not in colunas_elegibilidade:
            continue
        nifs_disp = sum(1 for nif, info in clientes_contactaveis.items() if campanha in info['elegiveis_para'])
        resultados.append({
            'campanha': campanha,
            'nifs_alocados': nifs_disp,
            'volumetria': nifs_disp,
            'offset': 0,
            'scenario': scenario_label
        })
    return pd.DataFrame(resultados, columns=['campanha','nifs_alocados','volumetria','offset','scenario'])

def alocar_plano(campanhas_df, scenario_label):
    df = campanhas_df.copy()
    df['nifs_alocados'] = df['volumetria']   # mantém volumetria e cria nifs_alocados
    df['offset'] = 0
    df['scenario'] = scenario_label
    return df[['campanha','nifs_alocados','volumetria','offset','scenario']]

# -------------------------------------------------------------------------
# 2.6 XS herdado (N1..N4 a partir de envios passados) + taxas multi
# -------------------------------------------------------------------------
exec_aloc = df_exec.groupby(['campanha','offset'], as_index=False)['nifs_alocados'].sum()
exec_aloc['volumetria'] = exec_aloc['nifs_alocados']

def aplicar_taxa_multi(df_aloc):
    df = df_aloc.merge(taxa_media, on=['campanha','offset'], how='left').fillna(0)
    df['montante_esperado']  = df['nifs_alocados'] * df['taxa_montante_mes']
    df['pedidos_esperado']   = df['nifs_alocados'] * df['taxa_pedidos_mes']
    df['aberturas_esperado'] = df['nifs_alocados'] * df['taxa_aberturas_mes']
    return df

exec_aloc = aplicar_taxa_multi(exec_aloc)

xs_herdado_montante  = exec_aloc[exec_aloc['offset'].between(1,4)]['montante_esperado'].sum()
xs_herdado_pedidos   = exec_aloc[exec_aloc['offset'].between(1,4)]['pedidos_esperado'].sum()
xs_herdado_aberturas = exec_aloc[exec_aloc['offset'].between(1,4)]['aberturas_esperado'].sum()

# -------------------------------------------------------------------------
# 2.6.1 XS MTD realista (mês corrente) com pace D0..D100  [ROBUSTO]
# -------------------------------------------------------------------------
import re

def normalizar_pace(df, col_camp='iTipoCampanha'):
    dias = [c for c in df.columns if re.match(r"^D\d+$", c)]
    dias = sorted(set(dias), key=lambda x: int(x[1:]))
    df = df[[col_camp] + dias].copy()
    df[dias] = df[dias].astype(float)
    soma = df[dias].sum(axis=1).replace(0, 1)
    df[dias] = df[dias].div(soma, axis=0)
    return df, dias

def aplicar_pace(df, pace_df, col_val, col_camp='iTipoCampanha'):
    tmp = df.merge(pace_df, on=col_camp, how='left')

    # garantir colunas únicas
    tmp = tmp.loc[:, ~tmp.columns.duplicated()].copy()

    # lista ordenada D0..D100 (apenas colunas D<num>)
    dias_cols = [c for c in tmp.columns if re.match(r"^D\d+$", c)]
    dias_cols = sorted(set(dias_cols), key=lambda x: int(x[1:]))

    # matriz de pace
    pace_mat = tmp[dias_cols].fillna(0).to_numpy()

    # dias passados desde envio até ontem
    tmp['dias_passados'] = (hoje.date() - tmp['DataInicio'].dt.date).apply(lambda x: x.days) - 1
    tmp['dias_passados'] = tmp['dias_passados'].clip(lower=0, upper=len(dias_cols)-1).astype(int)

    # pct maturada (cumsum e pick por índice)
    cumsum = pace_mat.cumsum(axis=1)
    idx = tmp['dias_passados'].to_numpy()
    tmp['pct_maturada'] = cumsum[np.arange(len(tmp)), idx]

    tmp[f'{col_val}_mtd'] = tmp[col_val] * tmp['pct_maturada']
    return tmp

# normalizar paces
pace_pedidos, _ = normalizar_pace(df_pace_pedidos)
pace_aberturas, _ = normalizar_pace(df_pace_aberturas)
pace_montante, _ = normalizar_pace(df_pace_montante)

# campanhas do mês atual
df_mes = df_resultados_campanha[
    (df_resultados_campanha['DataInicio'].dt.to_period('M') == pd.Timestamp(hoje).to_period('M'))
].copy()

# excluir campanhas do próprio dia
df_mes = df_mes[df_mes['DataInicio'].dt.date < hoje.date()]

# aplicar pace
df_mtd_ped = aplicar_pace(df_mes, pace_pedidos, 'QtdPedidos_N')
df_mtd_ab  = aplicar_pace(df_mes, pace_aberturas, 'QtdAberturas_N')
df_mtd_mon = aplicar_pace(df_mes, pace_montante, 'Montante_N')

# só campanhas XS (iXS_*)
df_mtd_ped_xs = df_mtd_ped[df_mtd_ped['iTipoCampanha'].str.startswith('iXS_', na=False)]
df_mtd_ab_xs  = df_mtd_ab[df_mtd_ab['iTipoCampanha'].str.startswith('iXS_', na=False)]
df_mtd_mon_xs = df_mtd_mon[df_mtd_mon['iTipoCampanha'].str.startswith('iXS_', na=False)]

xs_mtd_pedidos   = df_mtd_ped_xs['QtdPedidos_N_mtd'].sum()
xs_mtd_aberturas = df_mtd_ab_xs['QtdAberturas_N_mtd'].sum()
xs_mtd_montante  = df_mtd_mon_xs['Montante_N_mtd'].sum()



# -------------------------------------------------------------------------
# 2.7 Fidelização base (XS herdado + Outras XS + XS MTD realista)
# -------------------------------------------------------------------------
fidelizacao_base = xs_herdado_montante + fin_outras_xs + xs_mtd_montante

# -------------------------------------------------------------------------
# 2.8 Cenários (gap vs base)
# -------------------------------------------------------------------------
target_s1 = max(0, meta_fid - fidelizacao_base)
target_s2 = max(0, (meta_fid + gap_capt) - fidelizacao_base)

campanhas_atuais = df_fut[['campanha', 'volumetria']].copy()

res_s1 = aplicar_taxa_multi(alocar_por_target(campanhas_atuais, target_s1, "S1_FID"))
res_s2 = aplicar_taxa_multi(alocar_por_target(campanhas_atuais, target_s2, "S2_FID+GAP"))
res_s3 = aplicar_taxa_multi(alocar_todos(campanhas_atuais, "S3_TODOS"))
res_s4 = aplicar_taxa_multi(alocar_plano(campanhas_atuais, "S4_PLANO"))

# -------------------------------------------------------------------------
# 2.9 Executadas (fixas) e tabela comparativa
# -------------------------------------------------------------------------
def separar_prev_mes(df_exec, df_fut):
    prev = df_exec[df_exec['offset'] > 0].copy()
    mes  = df_fut[df_fut['offset'] == 0].copy()

    prev_grp = prev.groupby('campanha', as_index=False)[
        ['nifs_alocados','montante_esperado','pedidos_esperado','aberturas_esperado']
    ].sum()

    mes_grp = mes.groupby('campanha', as_index=False)[
        ['nifs_alocados','montante_esperado','pedidos_esperado','aberturas_esperado']
    ].sum()

    prev_grp.rename(columns={
        'nifs_alocados':'Envios_prev',
        'montante_esperado':'Montante_prev',
        'pedidos_esperado':'Pedidos_prev',
        'aberturas_esperado':'Aberturas_prev'
    }, inplace=True)

    mes_grp.rename(columns={
        'nifs_alocados':'Envios_mes',
        'montante_esperado':'Montante_mes',
        'pedidos_esperado':'Pedidos_mes',
        'aberturas_esperado':'Aberturas_mes'
    }, inplace=True)

    df = pd.merge(prev_grp, mes_grp, on='campanha', how='outer').fillna(0)

    df['Envios_total']     = df['Envios_prev'] + df['Envios_mes']
    df['Montante_total']   = df['Montante_prev'] + df['Montante_mes']
    df['Pedidos_total']    = df['Pedidos_prev'] + df['Pedidos_mes']
    df['Aberturas_total']  = df['Aberturas_prev'] + df['Aberturas_mes']
    return df

def construir_tabela(exec_df, fut_df, label):
    df = separar_prev_mes(exec_df, fut_df)
    df.columns = ['campanha'] + [f"{c}_{label}" for c in df.columns if c != 'campanha']
    return df

tab_s1 = construir_tabela(exec_aloc, res_s1, "S1")
tab_s2 = construir_tabela(exec_aloc, res_s2, "S2")
tab_s3 = construir_tabela(exec_aloc, res_s3, "S3")
tab_s4 = construir_tabela(exec_aloc, res_s4, "S4")

df_comparativo = tab_s1.merge(tab_s2, on='campanha', how='outer') \
                       .merge(tab_s3, on='campanha', how='outer') \
                       .merge(tab_s4, on='campanha', how='outer') \
                       .fillna(0)

df_comparativo = df_comparativo.sort_values('Montante_total_S4', ascending=False)

print(df_comparativo.to_string(index=False))

# -------------------------------------------------------------------------
# 2.10 Exportar Excel
# -------------------------------------------------------------------------
ficheiro = "./comparativo_cenarios.xlsx"
df_comparativo.to_excel(ficheiro, index=False)
print(f"Exportado para: {ficheiro}")

# -------------------------------------------------------------------------
# 2.11 Resumo final (contexto)
# -------------------------------------------------------------------------
print("\nRESUMO:")
print(f"- Dias úteis mês corrente: {dias_total}")
print(f"- Captação base (média 3M ajustada): {fin_capt:,.0f}")
print(f"- Outras XS base (média 3M ajustada): {fin_outras_xs:,.0f}")
print(f"- XS herdado (Montante): {xs_herdado_montante:,.0f}")
print(f"- XS herdado (Pedidos): {xs_herdado_pedidos:,.0f}")
print(f"- XS herdado (Aberturas): {xs_herdado_aberturas:,.0f}")
print(f"- XS MTD realista (Montante): {xs_mtd_montante:,.0f}")
print(f"- XS MTD realista (Pedidos): {xs_mtd_pedidos:,.0f}")
print(f"- XS MTD realista (Aberturas): {xs_mtd_aberturas:,.0f}")
print(f"- Fidelização base: {fidelizacao_base:,.0f}")
print(f"- Meta Fidelização: {meta_fid:,.0f}")
print(f"- Meta Captação: {meta_capt:,.0f}")
print(f"- Gap Captação: {gap_capt:,.0f}")
print(f"- Gap S1 (a cobrir): {target_s1:,.0f}")
print(f"- Gap S2 (a cobrir): {target_s2:,.0f}")



2. CENÁRIOS + RESUMO COMPARATIVO (com metas df_objetivos)

             campanha  Envios_prev_S1  Montante_prev_S1  Pedidos_prev_S1  Aberturas_prev_S1  Envios_mes_S1  Montante_mes_S1  Pedidos_mes_S1  Aberturas_mes_S1  Envios_total_S1  Montante_total_S1  Pedidos_total_S1  Aberturas_total_S1  Envios_prev_S2  Montante_prev_S2  Pedidos_prev_S2  Aberturas_prev_S2  Envios_mes_S2  Montante_mes_S2  Pedidos_mes_S2  Aberturas_mes_S2  Envios_total_S2  Montante_total_S2  Pedidos_total_S2  Aberturas_total_S2  Envios_prev_S3  Montante_prev_S3  Pedidos_prev_S3  Aberturas_prev_S3  Envios_mes_S3  Montante_mes_S3  Pedidos_mes_S3  Aberturas_mes_S3  Envios_total_S3  Montante_total_S3  Pedidos_total_S3  Aberturas_total_S3  Envios_prev_S4  Montante_prev_S4  Pedidos_prev_S4  Aberturas_prev_S4  Envios_mes_S4  Montante_mes_S4  Pedidos_mes_S4  Aberturas_mes_S4  Envios_total_S4  Montante_total_S4  Pedidos_total_S4  Aberturas_total_S4
  iXS_PCD_Inativos_1T             0.0               0.0              0.0      

: 

In [65]:
# =============================================================================
# BLOCO 3: CRIAÇÃO DE DICIONÁRIOS DE CLIENTES ELEGÍVEIS E CONTACTÁVEIS
# =============================================================================
print("\n3. CRIAÇÃO DE DICIONÁRIOS DE CLIENTES ELEGÍVEIS E CONTACTÁVEIS\n")

# Dicionário de elegibilidade: armazena as campanhas elegíveis para cada cliente
clientes_elegiveis = {}

# Criar o dicionário de clientes elegíveis baseado na #BaseEnvio
for _, row in df_baseenvio.iterrows():
    nif = row['NIF']
    elegiveis_para = []

    # Para cada campanha (coluna que começa com 'iXS_'), verificar elegibilidade
    for campanha in colunas_elegibilidade:
        if row[campanha] == 1:
            elegiveis_para.append(campanha)

    # Adicionar ao dicionário
    clientes_elegiveis[nif] = {
        'elegiveis_para': elegiveis_para,
        'num_elegiveis': len(elegiveis_para),
        # Identificar se o cliente é exclusivo de apenas uma campanha
        'exclusivo': len(elegiveis_para) == 1
    }

# Dicionário para armazenar clientes contactáveis
clientes_contactaveis = {}

# Definir as campanhas que não possuem bloqueio baseado no Mês Atual e Mês Anterior
CAMPANHAS_SEM_BLOQUEIO_MES_ANTERIOR = {'iXS_Terminados_', 'iXS_RUC_PlafondMinimo'}

# Verificar campanhas associadas ao cliente
def verificar_bloqueio(idmes, campanhas_cliente):
    """
    Verifica se um cliente está bloqueado por já ter recebido uma campanha no mês atual
    ou no mês anterior, exceto para as campanhas de exceção.
    """
    try:
        idmes = int(idmes)

        # Bloqueado sempre se recebeu campanha no mês atual
        if idmes == mes_atual:
            return True

        # Bloqueado para mês anterior (exceto campanhas específicas)
        if idmes == mes_anterior:
            return not any(c.startswith(tuple(CAMPANHAS_SEM_BLOQUEIO_MES_ANTERIOR)) for c in campanhas_cliente)

        return False  # Não está bloqueado
    except ValueError:
        return False  # Qualquer erro assume como 0 = Não bloqueado

# Criar dicionário de clients contactáveis
for nif, info in clientes_elegiveis.items():
    if info['elegiveis_para']:
        # Verificar se existem bloqueios para o cliente atual
        bloqueado = verificar_bloqueio(
            idmes=df_baseenvio.loc[df_baseenvio['NIF'] == nif, 'IdMesUltimaCampanha'].values[0],
            campanhas_cliente=info['elegiveis_para']
        )

        if not bloqueado:
            # Se não estiver bloqueado, adiciona ao dicionário de contactáveis
            clientes_contactaveis[nif] = {
                'elegiveis_para': info['elegiveis_para'],
                'num_elegiveis': info['num_elegiveis'],
                'exclusivo': info['exclusivo'],
                'dados_cliente': df_baseenvio.loc[df_baseenvio['NIF'] == nif].to_dict(orient='records')[0]
            }

# Dicionário para armazenar clientes exclusivos por campanha



3. CRIAÇÃO DE DICIONÁRIOS DE CLIENTES ELEGÍVEIS E CONTACTÁVEIS



In [66]:
# =============================================================================
# BLOCO 4: DISTRIBUIÇÃO OTIMIZADA DE CLIENTES PARA CAMPANHAS
# =============================================================================
print("\n4. DISTRIBUIÇÃO OTIMIZADA DE CLIENTES PARA CAMPANHAS ATUAIS\n")

# Inicializar estruturas para resultados e controles
resultados_distribuicao = []  # Lista para armazenar os resultados da distribuição
clientes_utilizados = set()  # Rastrear os NIFs já utilizados em campanhas
clientes_rejeitados = set()  # Rastrear NIFs alocados, mas não utilizados nesta rodada
contadores_campanhas = defaultdict(int)  # Contadores dos clientes por campanha

# Processar cada campanha do dia atual
if not campanhas_atuais.empty:
    print(f"✓ Campanhas a serem processadas: {len(campanhas_atuais):,}")

    for _, row in campanhas_atuais.iterrows():
        campanha = row['campanha']
        necessidade_total = int(row['volumetria'])  # Volumetria esperada para a campanha
        print(f"\n>> Processando Campanha: {campanha} (Necessidade: {necessidade_total:,})")

        # Obter todos os NIFs elegíveis para esta campanha que ainda não foram utilizados
        nifs_elegiveis = [
            nif for nif, info in clientes_contactaveis.items()
            if campanha in info['elegiveis_para'] and nif not in clientes_utilizados
        ]
        print(f"  • Clientes elegíveis disponíveis: {len(nifs_elegiveis):,}")

        if not nifs_elegiveis:
            print(f"  ⚠ Sem clientes elegíveis disponíveis para a campanha '{campanha}'")
            continue

        # -------------------- Seleção inicial de clientes --------------------
        # Priorizar clientes exclusivos
        exclusivos = [
            nif for nif in nifs_elegiveis
            if clientes_contactaveis[nif]['exclusivo'] and campanha in clientes_contactaveis[nif]['elegiveis_para']
        ]
        print(f"  • Clientes exclusivos desta campanha disponíveis: {len(exclusivos):,}")

        # Adicionar clientes exclusivos à distribuição prioritariamente
        nifs_selecionados = exclusivos[:necessidade_total]

        # Verificar se há mais espaço e selecionar clientes não exclusivos
        if len(nifs_selecionados) < necessidade_total:
            restantes_a_alocar = necessidade_total - len(nifs_selecionados)
            nao_exclusivos = [
                nif for nif in nifs_elegiveis
                if not clientes_contactaveis[nif]['exclusivo']
            ]
            # Adicionar os clientes não exclusivos restantes
            nifs_selecionados += nao_exclusivos[:restantes_a_alocar]

        # Garantir que não ultrapassamos o limite da volumetria
        nifs_selecionados = nifs_selecionados[:necessidade_total]

        print(f"  • Clientes selecionados: {len(nifs_selecionados):,} de {necessidade_total:,} necessários")

        # Adicionar clientes selecionados aos resultados e marcar como utilizados
        for nif in nifs_selecionados:
            cliente = clientes_contactaveis[nif]
            clientes_utilizados.add(nif)

            # Adicionar registro ao resultado
            resultados_distribuicao.append({
                'NIF': nif,
                'campanha': campanha,
                'prioridade': 'ALTA' if campanha == CAMPANHA_PRIORITARIA else 'NORMAL',
                'data_exportacao': hoje_str,
                'exclusivo': "SIM" if cliente['exclusivo'] else "NÃO",
                'num_elegiveis': cliente['num_elegiveis'],
                'campanhas_elegiveis': ', '.join(cliente['elegiveis_para']),
                'dados_cliente': cliente['dados_cliente']  # Adiciona todos os dados relacionados ao cliente
            })

        # Atualizar contadores da campanha
        contadores_campanhas[campanha] += len(nifs_selecionados)

        # Verificar e printar informações adicionais
        print(f"    ✓ Atribuídos com sucesso: {contadores_campanhas[campanha]:,}")

else:
    print("⚠ Nenhuma campanha para hoje.")

# =============================================================================
# RESUMO DA DISTRIBUIÇÃO
# =============================================================================
print("\nRESUMO DA DISTRIBUIÇÃO:")

if resultados_distribuicao:
    total_distribuidos = len(resultados_distribuicao)
    print(f"  • Total de clientes atribuídos hoje: {total_distribuidos:,}")

    for campanha, count in contadores_campanhas.items():
        print(f"  - Campanha '{campanha}': {count} clientes atribuídos.")

else:
    print("⚠ Nenhum cliente foi atribuído para campanhas de hoje.")



4. DISTRIBUIÇÃO OTIMIZADA DE CLIENTES PARA CAMPANHAS ATUAIS

✓ Campanhas a serem processadas: 1

>> Processando Campanha: iXS_PCDPM_Parcerias (Necessidade: 1,090)
  • Clientes elegíveis disponíveis: 13,175
  • Clientes exclusivos desta campanha disponíveis: 0
  • Clientes selecionados: 1,090 de 1,090 necessários
    ✓ Atribuídos com sucesso: 1,090

RESUMO DA DISTRIBUIÇÃO:
  • Total de clientes atribuídos hoje: 1,090
  - Campanha 'iXS_PCDPM_Parcerias': 1090 clientes atribuídos.


In [67]:
# =============================================================================
# BLOCO 5: SIMULAÇÃO FINAL: DISTRIBUIÇÃO DE CLIENTES E CAMPANHAS BASEADA NO HISTÓRICO
# =============================================================================
print("\n5. SIMULAÇÃO FINAL: DISTRIBUIÇÃO DE CLIENTES E CAMPANHAS BASEADA NO HISTÓRICO\n")

from collections import defaultdict

def calcular_similaridade_campanhas(df_baseenvio, campanhas_futuras):
    """
    Retorna a similaridade entre campanhas (interseção do público elegível).
    A similaridade é medida como a interseção de clientes elegíveis entre pares de campanhas.
    """
    similaridade = defaultdict(lambda: defaultdict(int))
    campanhas = campanhas_futuras['campanha'].tolist()

    for c1 in campanhas:
        for c2 in campanhas:
            if c1 != c2:
                elegiveis_c1 = set(df_baseenvio[df_baseenvio[c1] == 1]['NIF'])
                elegiveis_c2 = set(df_baseenvio[df_baseenvio[c2] == 1]['NIF'])
                similaridade[c1][c2] = len(elegiveis_c1.intersection(elegiveis_c2))

    return similaridade

def identificar_grupos_campanhas(campanhas):
    """
    Identifica os grupos de campanhas baseados em critérios:
    1. Campanhas com prefixo "CCR".
    2. Campanhas com o mesmo nome, mas diferentes por "_1T", "_2T", etc.
    """
    ccr_campanhas = [c for c in campanhas if c.startswith("CCR")]

    grupos_similares = defaultdict(list)
    for campanha in campanhas:
        if any(campanha.endswith(sufixo) for sufixo in ["1T", "2T", "3T"]):
            prefixo = "_".join(campanha.split("_")[:-1])
            grupos_similares[prefixo].append(campanha)

    return ccr_campanhas, grupos_similares

def obter_clientes_elegiveis(df_baseenvio, campanha):
    """
    Retorna os clientes elegíveis para uma campanha específica.
    """
    if campanha in df_baseenvio.columns:
        return df_baseenvio[df_baseenvio[campanha] == 1]['NIF'].tolist()
    return []

def obter_dias_uteis(data_inicio, data_fim):
    dias_uteis = []
    data_atual = data_inicio
    while data_atual <= data_fim:
        if data_atual.weekday() < 4:
            dias_uteis.append(data_atual)
        data_atual += timedelta(days=1)
    return dias_uteis

# =============================================================================
# INICIAR SIMULAÇÃO
# =============================================================================
primeiro_dia_mes = datetime(hoje.year, hoje.month, 1)
ultimo_dia_mes = (primeiro_dia_mes + timedelta(days=31)).replace(day=1) - timedelta(days=1)
dias_uteis_mes = obter_dias_uteis(primeiro_dia_mes, ultimo_dia_mes)
dias_ate_85_porcento = dias_uteis_mes[:int(len(dias_uteis_mes) * 0.85)]

print(f"✓ Dias úteis disponíveis nos primeiros 80% do mês: {len(dias_ate_85_porcento)}")
print(f"  [{', '.join([d.strftime('%d/%m/%Y') for d in dias_ate_85_porcento])}]")

ccr_campanhas, grupos_similares = identificar_grupos_campanhas(campanhas_futuras['campanha'].tolist())
similaridade_campanhas = calcular_similaridade_campanhas(df_baseenvio, campanhas_futuras)

print(f"\n✓ Campanhas CCR identificadas para envio conjunto: {ccr_campanhas}")
print(f"✓ Identificados grupos de campanhas semelhantes por sufixo: {grupos_similares}")

campanhas_por_dia = {dia: [] for dia in dias_uteis_mes}
clientes_atribuidos = set()
sugestoes_dias = []

# Alocar campanhas CCR para o mesmo dia
if ccr_campanhas:
    ccr_dia = dias_uteis_mes[0]
    for campanha in ccr_campanhas:
        clientes_elegiveis = obter_clientes_elegiveis(df_baseenvio, campanha)
        clientes_disponiveis = [nif for nif in clientes_elegiveis if nif not in clientes_atribuidos]
        volumetria_original = campanhas_futuras[campanhas_futuras['campanha'] == campanha]['volumetria'].values[0]
        volumetria_sugerida = min(len(clientes_disponiveis), volumetria_original)

        clientes_atribuidos.update(clientes_disponiveis[:volumetria_sugerida])

        campanhas_por_dia[ccr_dia].append(campanha)
        sugestoes_dias.append({
            'Campanha': campanha,
            'Data Sugerida': ccr_dia.strftime('%d/%m/%Y'),
            'Volumetria Original': volumetria_original,
            'Volumetria Sugerida': volumetria_sugerida,
            'Elegíveis Restantes': len(clientes_disponiveis) - volumetria_sugerida
        })

# Alocar grupos "_1T", "_2T" para o mesmo dia
for prefixo, grupo in grupos_similares.items():
    dia_grupo = dias_uteis_mes[1]
    for campanha in grupo:
        clientes_elegiveis = obter_clientes_elegiveis(df_baseenvio, campanha)
        clientes_disponiveis = [nif for nif in clientes_elegiveis if nif not in clientes_atribuidos]
        volumetria_original = campanhas_futuras[campanhas_futuras['campanha'] == campanha]['volumetria'].values[0]
        volumetria_sugerida = min(len(clientes_disponiveis), volumetria_original)

        clientes_atribuidos.update(clientes_disponiveis[:volumetria_sugerida])

        campanhas_por_dia[dia_grupo].append(campanha)
        sugestoes_dias.append({
            'Campanha': campanha,
            'Data Sugerida': dia_grupo.strftime('%d/%m/%Y'),
            'Volumetria Original': volumetria_original,
            'Volumetria Sugerida': volumetria_sugerida,
            'Elegíveis Restantes': len(clientes_disponiveis) - volumetria_sugerida
        })

# Distribuir campanhas restantes
for _, linha in campanhas_futuras.iterrows():
    campanha = linha['campanha']
    if campanha in ccr_campanhas or any(c for group in grupos_similares.values() for c in group if c == campanha):
        continue

    for dia_sugerido in dias_ate_85_porcento:
        if not any(similaridade_campanhas[campanha][c] > 0 for c in campanhas_por_dia[dia_sugerido]):
            clientes_elegiveis = obter_clientes_elegiveis(df_baseenvio, campanha)
            clientes_disponiveis = [nif for nif in clientes_elegiveis if nif not in clientes_atribuidos]
            volumetria_original = linha['volumetria']
            volumetria_sugerida = min(len(clientes_disponiveis), volumetria_original)

            clientes_selecionados = clientes_disponiveis[:volumetria_sugerida]
            clientes_atribuidos.update(clientes_selecionados)
            campanhas_por_dia[dia_sugerido].append(campanha)

            sugestoes_dias.append({
                'Campanha': campanha,
                'Data Sugerida': dia_sugerido.strftime('%d/%m/%Y'),
                'Volumetria Original': volumetria_original,
                'Volumetria Sugerida': volumetria_sugerida,
                'Elegíveis Restantes': len(clientes_disponiveis) - volumetria_sugerida
            })
            break

# Converter planejamento para DataFrame
df_sugestao_dias = pd.DataFrame(sugestoes_dias)



5. SIMULAÇÃO FINAL: DISTRIBUIÇÃO DE CLIENTES E CAMPANHAS BASEADA NO HISTÓRICO

✓ Dias úteis disponíveis nos primeiros 80% do mês: 13
  [02/02/2026, 03/02/2026, 04/02/2026, 05/02/2026, 09/02/2026, 10/02/2026, 11/02/2026, 12/02/2026, 16/02/2026, 17/02/2026, 18/02/2026, 19/02/2026, 23/02/2026]

✓ Campanhas CCR identificadas para envio conjunto: []
✓ Identificados grupos de campanhas semelhantes por sufixo: defaultdict(<class 'list'>, {'iXS_Terminados_PA': ['iXS_Terminados_PA_2T'], 'iXS_PCD_Inativos': ['iXS_PCD_Inativos_2T', 'iXS_PCD_Inativos_1T']})


In [68]:
# =============================================================================
# BLOCO 6: EXPORTAÇÃO FINAL DOS RESULTADOS (COM CORREÇÕES)
# =============================================================================
print("\n6. EXPORTAÇÃO FINAL: GERAÇÃO DE RELATÓRIOS E RESULTADOS DE CAMPANHAS\n")

def calcular_clientes_exclusivos_por_campanha(df_baseenvio, campanhas_futuras):
    colunas_campanhas = campanhas_futuras['campanha'].tolist()

    for coluna in colunas_campanhas:
        if coluna not in df_baseenvio.columns:
            raise KeyError(f"⚠️ A coluna {coluna} não foi encontrada em 'df_baseenvio'. Verifique a integridade dos dados.")

    df_campanhas = df_baseenvio[['NIF'] + colunas_campanhas]
    df_campanhas['num_elegiveis'] = df_campanhas[colunas_campanhas].sum(axis=1)

    clientes_exclusivos_dic = {}
    for campanha in colunas_campanhas:
        clientes_exclusivos = df_campanhas[
            (df_campanhas['num_elegiveis'] == 1) & (df_campanhas[campanha] == 1)
        ]['NIF'].tolist()
        clientes_exclusivos_dic[campanha] = clientes_exclusivos

    return clientes_exclusivos_dic

if 'campanhas_futuras' not in locals() or 'df_baseenvio' not in locals():
    raise ValueError("⚠️ Variáveis 'campanhas_futuras' e/ou 'df_baseenvio' não estão definidas. Execute os blocos anteriores corretamente.")

clientes_exclusivos_por_campanha = calcular_clientes_exclusivos_por_campanha(df_baseenvio, campanhas_futuras)
print("\n✓ Clientes exclusivos por campanha calculados.\n")

def gerar_resumo_por_campanha(campanhas_futuras, resultados_distribuicao, clientes_exclusivos_por_campanha):
    if 'Campanha' not in resultados_distribuicao.columns:
        raise KeyError("⚠️ A coluna 'Campanha' está ausente no DataFrame 'resultados_distribuicao'.")

    resumo_campanhas = []
    for _, row in campanhas_futuras.iterrows():
        campanha = row['campanha']
        necessidade_total = row['volumetria']
        total_atribuido = resultados_distribuicao[resultados_distribuicao['Campanha'] == campanha].shape[0]
        clientes_exclusivos = len(clientes_exclusivos_por_campanha.get(campanha, []))

        resumo_campanhas.append({
            'Campanha': campanha,
            'Volumetria Planejada': necessidade_total,
            'Total Atribuído': total_atribuido,
            'Percentual de Atendimento (%)': round((total_atribuido / necessidade_total * 100), 2) if necessidade_total > 0 else 0,
            'Clientes Exclusivos Usados': clientes_exclusivos
        })

    return resumo_campanhas

if 'df_sugestao_dias' not in locals():
    raise ValueError("⚠️ O dataframe df_sugestao_dias não está definido. Execute o Bloco 5 antes de prosseguir!")

resultados_distribuicao = df_sugestao_dias
print("\n✓ Resultados da distribuição carregados com sucesso.")

resumo_campanhas = gerar_resumo_por_campanha(campanhas_futuras, resultados_distribuicao, clientes_exclusivos_por_campanha)
print("\n✓ Resumo por campanha gerado.")

df_resumo_campanhas = pd.DataFrame(resumo_campanhas)
arquivo_resumo_campanhas = os.path.join(FOLDER_OUTPUT, f"resumo_campanhas_{hoje_file}.csv")
df_resumo_campanhas.to_csv(arquivo_resumo_campanhas, sep=SEPARADOR, encoding=ENCODING, index=False)
print(f"\n✓ Resumo por campanhas salvo em: {arquivo_resumo_campanhas}")

for campanha in resultados_distribuicao['Campanha'].unique():
    clientes_campanha = resultados_distribuicao[resultados_distribuicao['Campanha'] == campanha]
    pasta_campanha = os.path.join(FOLDER_OUTPUT, f"Campanha_{campanha.replace(' ', '_')}")
    os.makedirs(pasta_campanha, exist_ok=True)

    arquivo_campanha = os.path.join(pasta_campanha, f"{campanha}_clientes.csv")
    clientes_campanha.to_csv(arquivo_campanha, sep=";", encoding="utf-8", index=False)
    print(f"✓ Relatório para '{campanha}' salvo com sucesso em {arquivo_campanha}")

print("\n✓ Exportação completa: Resumo e relatórios detalhados.")



6. EXPORTAÇÃO FINAL: GERAÇÃO DE RELATÓRIOS E RESULTADOS DE CAMPANHAS


✓ Clientes exclusivos por campanha calculados.


✓ Resultados da distribuição carregados com sucesso.

✓ Resumo por campanha gerado.

✓ Resumo por campanhas salvo em: C:\Users\COTAGO\Desktop\Base\Export_Campanhas\resumo_campanhas_20260216_163057.csv
✓ Relatório para 'iXS_Terminados_PA_2T' salvo com sucesso em C:\Users\COTAGO\Desktop\Base\Export_Campanhas\Campanha_iXS_Terminados_PA_2T\iXS_Terminados_PA_2T_clientes.csv
✓ Relatório para 'iXS_PCD_Inativos_2T' salvo com sucesso em C:\Users\COTAGO\Desktop\Base\Export_Campanhas\Campanha_iXS_PCD_Inativos_2T\iXS_PCD_Inativos_2T_clientes.csv
✓ Relatório para 'iXS_PCD_Inativos_1T' salvo com sucesso em C:\Users\COTAGO\Desktop\Base\Export_Campanhas\Campanha_iXS_PCD_Inativos_1T\iXS_PCD_Inativos_1T_clientes.csv
✓ Relatório para 'iXS_RUC_PlafondMinimo' salvo com sucesso em C:\Users\COTAGO\Desktop\Base\Export_Campanhas\Campanha_iXS_RUC_PlafondMinimo\iXS_RUC_PlafondMinimo_clientes.c

In [87]:
# =============================================================================
# BLOCO 7: FILTRAR CAMPANHAS DE HOJE
# =============================================================================
print("\n7. FILTRAR CAMPANHAS DE HOJE")

if 'hoje_str' in globals():
    hoje_data = pd.to_datetime(hoje_str).date()
else:
    hoje_data = datetime.now().date()

campanhas_futuras['data'] = pd.to_datetime(campanhas_atuais['data']).dt.date
campanhas_hoje = campanhas_atuais[campanhas_atuais['data'] == hoje_str].copy()

print(f"✓ Campanhas de hoje: {len(campanhas_hoje)}")
if len(campanhas_hoje) > 0:
    print(campanhas_hoje[['campanha', 'volumetria', 'data']].to_string(index=False))



7. FILTRAR CAMPANHAS DE HOJE
✓ Campanhas de hoje: 1
           campanha  volumetria       data
iXS_PCDPM_Parcerias        1090 2026-02-16


In [96]:
# RECRIAR clientes_elegiveis corretamente (dict)
clientes_elegiveis = {}

for _, row in df_baseenvio.iterrows():
    nif = row['NIF']
    elegiveis_para = [c for c in colunas_elegibilidade if row[c] == 1]

    clientes_elegiveis[nif] = {
        'elegiveis_para': elegiveis_para,
        'num_elegiveis': len(elegiveis_para),
        'exclusivo': len(elegiveis_para) == 1
    }


In [109]:
# =============================================================================
# BLOCO 8: PROCESSAR CAMPANHAS DE HOJE (ATUALIZADO)
# =============================================================================
print("\n8. PROCESSAR CAMPANHAS DE HOJE")

resultados = []
clientes_utilizados = set()
contadores_campanhas = defaultdict(int)

MAPEAMENTO_TIPO_OFERTA = {
    'PCD': 'Melhor_Match_PCD',
    'RUC': 'Melhor_Match_RUC'
}


def calcular_valor_cliente(cliente_info, campanha, detalhes_escassez, clientes_elegiveis):
    # prioridade simples: meses sem contato + peso de escassez
    meses = cliente_info.get('meses_sem_contato', 0)
    esc = detalhes_escassez.get(campanha, {}).get('escassez', 0)
    return meses + (esc * 100)


if len(campanhas_hoje) > 0:
    campanhas_hoje = campanhas_hoje.sort_values('data')
    print(f"✓ Campanhas para processar hoje: {len(campanhas_hoje)}")

    for _, row in campanhas_hoje.iterrows():
        campanha = row['campanha']
        necessidade = int(row['volumetria'])
        tipo_oferta = row.get('tipo oferta', None)
        ofertafixa = row.get('ofertafixa', None)

        print(f"\n  Processando {campanha}:")
        print(f"    • Necessita: {necessidade:,} clientes")

        if campanha not in colunas_elegibilidade:
            print(f" Campanha não encontrada nos elegíveis")
            continue

        nifs_elegiveis = [
            nif for nif, info in clientes_elegiveis.items()
            if campanha in info['elegiveis_para'] and nif not in clientes_utilizados
        ]

        print(f"Elegíveis disponíveis: {len(nifs_elegiveis):,}")

        if len(nifs_elegiveis) == 0:
            print(f"Sem elegíveis disponíveis para esta campanha")
            continue

        quantidade_a_atribuir = min(necessidade, len(nifs_elegiveis))

        if quantidade_a_atribuir < necessidade:
            print(f"ATENÇÃO: Apenas {quantidade_a_atribuir:,} disponíveis de {necessidade:,} necessários")
            print(f"    • Atribuindo todos os {quantidade_a_atribuir:,} disponíveis")
        else:
            print(f"    • Atribuindo {quantidade_a_atribuir:,} de {necessidade:,} necessários")

        valores_clientes = []
        for nif in nifs_elegiveis:
            cliente_idx = df_baseenvio[df_baseenvio['NIF'] == nif].index
            if len(cliente_idx) > 0:
                cliente_row = df_baseenvio.loc[cliente_idx[0]]
                cliente_info = {
                    'NIF': nif,
                    'meses_sem_contato': cliente_row.get('meses_sem_contato', 999)
                }

                valor = calcular_valor_cliente(
                    cliente_info,
                    campanha,
                    detalhes_escassez,
                    clientes_elegiveis
                )

                valores_clientes.append((nif, valor))

        valores_clientes.sort(key=lambda x: x[1], reverse=True)
        melhores_clientes = valores_clientes[:quantidade_a_atribuir]

        print(f"    • Selecionados {len(melhores_clientes):,} melhores clientes")

        for nif, valor in melhores_clientes:
            cliente_idx = df_baseenvio[df_baseenvio['NIF'] == nif].index
            if len(cliente_idx) == 0:
                continue

            cliente_row = df_baseenvio.loc[cliente_idx[0]]

            opt_out_email = False
            opt_out_sms = False

            if 'iOptOut_Email' in cliente_row:
                val = str(cliente_row['iOptOut_Email']).upper().strip()
                opt_out_email = val in ['VERDADEIRO', 'TRUE', '1', 'SIM']

            if 'iOptOut_SMS' in cliente_row:
                val = str(cliente_row['iOptOut_SMS']).upper().strip()
                opt_out_sms = val in ['VERDADEIRO', 'TRUE', '1', 'SIM']

            canais = []
            if not opt_out_email:
                canais.append('EMAIL')
            if not opt_out_sms:
                canais.append('SMS')

            info_cliente = clientes_elegiveis[nif]
            exclusivo = info_cliente['exclusivo']
            campanha_exclusiva = info_cliente['elegiveis_para'][0] if exclusivo else None

            campanhas_criticas_futuras = []
            for camp_fut in info_cliente['elegiveis_para']:
                if camp_fut != campanha and camp_fut in detalhes_escassez:
                    if detalhes_escassez[camp_fut]['escassez'] > 0.5:
                        campanhas_criticas_futuras.append(camp_fut)

            amount_exportar = None
            if pd.notna(ofertafixa):
                amount_exportar = ofertafixa

            coluna_match = None
            tipo_oferta_campanha = row.get('tipo oferta', None)
            if tipo_oferta_campanha:
                coluna_match = MAPEAMENTO_TIPO_OFERTA.get(tipo_oferta_campanha)

            if coluna_match and coluna_match in cliente_row:
                amount_exportar = cliente_row[coluna_match]

            resultados.append({
                'NIF': nif,
                'campanha': campanha,
                'prioridade': 'ALTA' if campanha == CAMPANHA_PRIORITARIA else 'NORMAL',
                'data_exportacao': hoje_str,
                'canais': '|'.join(canais),
                'opt_out_email': 'SIM' if opt_out_email else 'NÃO',
                'opt_out_sms': 'SIM' if opt_out_sms else 'NÃO',
                'meses_sem_contato': cliente_row.get('meses_sem_contato', 'N/A'),
                'exclusivo_desta': 'SIM' if exclusivo and campanha_exclusiva == campanha else 'NÃO',
                'valor_atribuicao': valor,
                'campanhas_criticas_futuras': ','.join(campanhas_criticas_futuras) if campanhas_criticas_futuras else 'NENHUMA',
                'AMOUNT': amount_exportar
            })

            clientes_utilizados.add(nif)
            contadores_campanhas[campanha] += 1

        print(f"    ✓ Atribuídos: {contadores_campanhas[campanha]:,}")

        if contadores_campanhas[campanha] > 0:
            atribuidos_campanha = [r for r in resultados if r['campanha'] == campanha]
            exclusivos = len([r for r in atribuidos_campanha if r['exclusivo_desta'] == 'SIM'])
            com_criticas = len([r for r in atribuidos_campanha if r['campanhas_criticas_futuras'] != 'NENHUMA'])

            print(f"      • Exclusivos desta campanha: {exclusivos:,}")
            print(f"      • Também elegíveis para campanhas críticas futuras: {com_criticas:,}")

            if com_criticas > 0 and detalhes_escassez.get(campanha, {}).get('escassez', 0) < 0.3:
                print(f"      ⚠ ATENÇÃO: Usando clientes valiosos para campanha não crítica")
else:
    print("  ⚠ Nenhuma campanha para hoje")



8. PROCESSAR CAMPANHAS DE HOJE
✓ Campanhas para processar hoje: 1

  Processando iXS_PCDPM_Parcerias:
    • Necessita: 1,090 clientes
Elegíveis disponíveis: 45,449
    • Atribuindo 1,090 de 1,090 necessários
    • Selecionados 1,090 melhores clientes
    ✓ Atribuídos: 1,090
      • Exclusivos desta campanha: 0
      • Também elegíveis para campanhas críticas futuras: 1
      ⚠ ATENÇÃO: Usando clientes valiosos para campanha não crítica


In [110]:
# =============================================================================
# BLOCO 9: ANÁLISE DA DISTRIBUIÇÃO (ATUALIZADO)
# =============================================================================
print("\n9. ANÁLISE DA DISTRIBUIÇÃO")

if resultados:
    print(f"\n✓ Total clientes atribuídos: {len(resultados):,}")

    print(f"\n✓ Atendimento das necessidades:")
    total_necessidade = 0
    total_atribuido = 0

    for _, row in campanhas_hoje.iterrows():
        campanha = row['campanha']
        necessidade = int(row['volumetria'])
        atribuidos = contadores_campanhas.get(campanha, 0)

        total_necessidade += necessidade
        total_atribuido += atribuidos

        if necessidade > 0:
            percentual = (atribuidos / necessidade) * 100 if necessidade > 0 else 0
            status = "✓ COMPLETO" if atribuidos >= necessidade else f"✓ PARCIAL ({percentual:.1f}%)"
            if atribuidos == 0:
                status = "⚠ NENHUM"

            print(f"  • {campanha}: {atribuidos:,} de {necessidade:,} - {status}")

    print(f"\n✓ Resumo Geral:")
    print(f"  • Necessidade total: {total_necessidade:,} clientes")
    print(f"  • Atribuídos: {total_atribuido:,} clientes")
    print(f"  • Atendimento: {(total_atribuido/total_necessidade*100 if total_necessidade > 0 else 0):.1f}%")

    exclusivos = len([r for r in resultados if r['exclusivo_desta'] == 'SIM'])
    nao_exclusivos = len(resultados) - exclusivos

    print(f"\n✓ Distribuição por tipo:")
    print(f"  • Clientes exclusivos usados: {exclusivos:,} ({exclusivos/len(resultados)*100:.1f}%)")
    print(f"  • Clientes não exclusivos usados: {nao_exclusivos:,} ({nao_exclusivos/len(resultados)*100:.1f}%)")

    print(f"\n✓ Impacto nas campanhas futuras críticas:")

    for campanha, detalhes in detalhes_escassez.items():
        if detalhes['escassez'] > 0.5:
            elegiveis_campanha = [nif for nif, info in clientes_elegiveis.items() if campanha in info['elegiveis_para']]
            usados_hoje = len([nif for nif in elegiveis_campanha if nif in clientes_utilizados])

            if usados_hoje > 0:
                percent_usados = usados_hoje / detalhes['elegiveis_totais'] * 100
                print(f"  • {campanha} ({detalhes['data']}):")
                print(f"    Usados hoje: {usados_hoje:,} de {detalhes['elegiveis_totais']:,} ({percent_usados:.1f}%)")

                if percent_usados > 10:
                    print(f"    ⚠ ALTA UTILIZAÇÃO para campanha crítica futura!")



9. ANÁLISE DA DISTRIBUIÇÃO

✓ Total clientes atribuídos: 1,090

✓ Atendimento das necessidades:
  • iXS_PCDPM_Parcerias: 1,090 de 1,090 - ✓ COMPLETO

✓ Resumo Geral:
  • Necessidade total: 1,090 clientes
  • Atribuídos: 1,090 clientes
  • Atendimento: 100.0%

✓ Distribuição por tipo:
  • Clientes exclusivos usados: 0 (0.0%)
  • Clientes não exclusivos usados: 1,090 (100.0%)

✓ Impacto nas campanhas futuras críticas:
  • iXS_RUC_PlafondMinimo (24/02/2026):
    Usados hoje: 1 de 157 (0.6%)


In [111]:
# =============================================================================
# BLOCO 10: EXPORTAR RESULTADOS PARA PCD E RUC (VERSÃO FINAL)
# =============================================================================
print("\n10. EXPORTAR RESULTADOS")

import pandas as pd
import numpy as np
import os
import pyodbc
from datetime import datetime, timedelta

validade_camp = (datetime.now() + timedelta(days=30)).strftime('%Y-%m-%d')

# -------------------------------------------------------------------------
# COLEGAS
# -------------------------------------------------------------------------
COLEGAS = [
    {'param2': '100000001', 'external_id': '100000001', 'Email': 'anamrito@gmail.com', 'phone': '916361416', 'name': 'Ana', 'iSecurizado': 1, 'iApp': 1},
    {'param2': '100000002', 'external_id': '100000002', 'Email': 'marco.aureliorodrigues@cofidis.pt', 'phone': '910827450', 'name': 'Marco', 'iSecurizado': 1, 'iApp': 1},
    {'param2': '100000003', 'external_id': '100000003', 'Email': 'sonia.fonseca@cofidis.pt', 'phone': '910824629', 'name': 'Sonia', 'iSecurizado': 1, 'iApp': 1},
    {'param2': '100000004', 'external_id': '100000004', 'Email': 'daniel.antonio@cofidis.pt', 'phone': '919972772', 'name': 'Daniel', 'iSecurizado': 1, 'iApp': 1},
    {'param2': '100000005', 'external_id': '100000005', 'Email': 'catarina.rodrigues@cofidis.pt', 'phone': '910835766', 'name': 'Catarina', 'iSecurizado': 1, 'iApp': 1},
    {'param2': '100000006', 'external_id': '100000006', 'Email': 'etienne.clause@cofidis.pt', 'phone': '937541148', 'name': 'Etienne', 'iSecurizado': 1, 'iApp': 1},
    {'param2': '100000007', 'external_id': '100000007', 'Email': 'cinthya.pozzer@cofidis.pt', 'phone': '915902254', 'name': 'Cinthya', 'iSecurizado': 1, 'iApp': 1},
    {'param2': '100000008', 'external_id': '100000008', 'Email': 'josedjusto@gmail.com', 'phone': '966663962', 'name': 'Jose', 'iSecurizado': 1, 'iApp': 1},
    {'param2': '100000010', 'external_id': '100000010', 'Email': 'goncalo.cota@cofidis.pt', 'phone': '962951553', 'name': 'Goncalo', 'iSecurizado': 1, 'iApp': 1},
    {'param2': '100000011', 'external_id': '100000011', 'Email': 'gustavo.baptista@cofidis.pt', 'phone': '910126016', 'name': 'Gustavo', 'iSecurizado': 1, 'iApp': 1},
    {'param2': '100000012', 'external_id': '100000012', 'Email': 'rita@yoomen.pt', 'phone': '925483420', 'name': 'Rita', 'iSecurizado': 1, 'iApp': 1},
    {'param2': '100000013', 'external_id': '100000013', 'Email': 'rui.agriao@cofidis.pt', 'phone': '965463660', 'name': 'Rui', 'iSecurizado': 1, 'iApp': 1}
]

# -------------------------------------------------------------------------
# SQL HELPERS
# -------------------------------------------------------------------------
def get_sql_connection():
    try:
        return pyodbc.connect(
            'Driver={SQL Server};'
            'Server=Diomedes;'
            'Database=tempdb;'
            'Trusted_Connection=yes;'
        )
    except:
        return None

def obter_proposta(tipo):
    conn = get_sql_connection()
    if conn is None:
        return pd.DataFrame()

    if tipo == "PCD":
        query = """
        SELECT DISTINCT
            AMOUNT,
            tan AS RATE,
            TAEG AS APR,
            DEADLINE,
            MONTHLYPAYWITHINSURANCE AS MONTHLYPAY,
            MONTHLYPAYWITHOUTINSURANCE,
            MONTHLYPAYWITHINSURANCE - MONTHLYPAYWITHOUTINSURANCE AS INSURANCE,
            MTIC
        FROM (
            SELECT a.*, ROW_NUMBER() OVER (PARTITION BY AMOUNT ORDER BY UpdatedDate DESC) AS rn
            FROM armada.dbo.ST_DM_CondicoesSimulacao a
            LEFT JOIN armada.dbo.DM_Oferta b ON a.idoffer = b.idOferta
            LEFT JOIN armada.dbo.IDH_Dim_Produto c ON b.Produto = c.CodProdutoDAH
            WHERE a.AMOUNT IN (2500,3000,3500,4000,4500,5000,6000,6500,7000,7500,8000)
              AND DEADLINEWITHINSURANCE = 84 AND PURPOSEID = 6
        ) x WHERE rn = 1 ORDER BY AMOUNT
        """
    else:
        query = """
        SELECT DISTINCT
            AMOUNT,
            tan AS RATE,
            TAEG AS APR,
            DEADLINE,
            MonthlyPayWithInsurance AS MONTHLYPAY,
            CASE WHEN AMOUNT=1000 THEN 7 WHEN AMOUNT=1500 THEN 9
                 WHEN AMOUNT=2000 THEN 14 WHEN AMOUNT=3000 THEN 21
                 WHEN AMOUNT=4000 THEN 28 ELSE 0 END AS INSURANCE,
            MTIC
        FROM (
            SELECT a.*, ROW_NUMBER() OVER (PARTITION BY AMOUNT ORDER BY UpdatedDate DESC) AS rn
            FROM armada.dbo.ST_DM_CondicoesSimulacao a
            LEFT JOIN armada.dbo.DM_Oferta b ON a.idoffer = b.idOferta
            LEFT JOIN armada.dbo.IDH_Dim_Produto c ON b.Produto = c.CodProdutoDAH
            WHERE c.CodProduto='RUC' AND a.AMOUNT IN (1000,1500,2000,3000,4000)
        ) x WHERE rn = 1 ORDER BY AMOUNT
        """

    df = pd.read_sql(query, conn)
    df['AMOUNT'] = df['AMOUNT'].astype(int)
    return df

def obter_dossiers():
    conn = get_sql_connection()
    if conn is None:
        return pd.DataFrame()

    query = """
    SELECT DISTINCT a1.nif, a1.numdossier
    FROM armada.dbo.dm_clienteprocesso a1
    INNER JOIN armada.dbo.dm_dossier a2 ON a1.numdossier=a2.numdossier AND idossieremcurso=1
    WHERE TipoInterveniente='T' AND ordem=1
    """
    return pd.read_sql(query, conn)

# -------------------------------------------------------------------------
# FUNÇÃO PRINCIPAL
# -------------------------------------------------------------------------
def processar_campanha(campanha_nome, df_clientes, necessidade, tipo, df_res):
    print(f"\n  Processando campanha {tipo}: {campanha_nome}")

    if df_clientes.empty:
        return None, None, None

    qtd = min(necessidade, len(df_clientes))
    df_clientes = df_clientes.head(qtd).copy()

    df_clientes = df_clientes.merge(df_res[['NIF', 'AMOUNT']], on='NIF', how='left')
    df_clientes['AMOUNT'] = pd.to_numeric(df_clientes['AMOUNT'], errors='coerce').fillna(0).astype(int)

    df_prop = obter_proposta(tipo)
    df_full = df_clientes.merge(df_prop, on='AMOUNT', how='left')

    if tipo == "PCD":
        df_cont = pd.DataFrame({
            'param2': df_full['NIF'],
            'name': df_full.get('PrimNome', ''),
            'Email': df_full.get('Email', ''),
            'phone': df_full.get('telemovel', ''),
            'param6': df_full['MONTHLYPAYWITHOUTINSURANCE'],
            'param7': df_full['AMOUNT'],
            'param8': df_full['DEADLINE'],
            'param10': df_full['APR'],
            'param11': df_full['RATE'],
            'param12': df_full['MONTHLYPAY'],
            'param16': validade_camp,
            'param17': df_full['INSURANCE'],
            'param18': df_full['MONTHLYPAY'],
            'param19': df_full['MTIC'],
            'external_id': df_full['NIF'],
            'iSecurizado': 1,
            'iApp': 1,
            'Ordem': 1
        })
    else:
        df_cont = pd.DataFrame({
            'param2': df_full['NIF'],
            'name': df_full.get('PrimNome', ''),
            'Email': df_full.get('Email', ''),
            'phone': df_full.get('telemovel', ''),
            'param6': df_full['MONTHLYPAY'],
            'param7': df_full['AMOUNT'],
            'param8': df_full['DEADLINE'],
            'param9': df_full['INSURANCE'],
            'param10': df_full['APR'],
            'param11': df_full['RATE'],
            'param12': df_full['MONTHLYPAY'],
            'param16': validade_camp,
            'external_id': df_full['NIF'],
            'iSecurizado': 1,
            'iApp': 1,
            'Ordem': 1
        })

    df_colegas = pd.DataFrame(COLEGAS)
    maior_montante = df_prop['AMOUNT'].max()
    linha_max = df_prop[df_prop['AMOUNT'] == maior_montante].iloc[0]

    if tipo == "PCD":
        df_colegas['param6'] = linha_max['MONTHLYPAYWITHOUTINSURANCE']
        df_colegas['param7'] = linha_max['AMOUNT']
        df_colegas['param8'] = linha_max['DEADLINE']
        df_colegas['param10'] = linha_max['APR']
        df_colegas['param11'] = linha_max['RATE']
        df_colegas['param12'] = linha_max['MONTHLYPAY']
        df_colegas['param16'] = validade_camp
        df_colegas['param17'] = linha_max['INSURANCE']
        df_colegas['param18'] = linha_max['MONTHLYPAY']
        df_colegas['param19'] = linha_max['MTIC']
        df_colegas['Ordem'] = 1
    else:
        df_colegas['param6'] = linha_max['MONTHLYPAY']
        df_colegas['param7'] = linha_max['AMOUNT']
        df_colegas['param8'] = linha_max['DEADLINE']
        df_colegas['param9'] = linha_max['INSURANCE']
        df_colegas['param10'] = linha_max['APR']
        df_colegas['param11'] = linha_max['RATE']
        df_colegas['param12'] = linha_max['MONTHLYPAY']
        df_colegas['param16'] = validade_camp
        df_colegas['Ordem'] = 1

    df_cont = pd.concat([df_cont, df_colegas], ignore_index=True)
    df_cont = df_cont.replace(['None', 'nan', 'NaN'], np.nan)

    if 'phone' in df_cont.columns:
        df_cont['phone'] = df_cont['phone'].apply(
            lambda x: '' if pd.isna(x) or str(x).strip() == '' else str(int(float(x)))
        )

    cols_int = ['param2','param7','param8','param9','param11','external_id','iSecurizado','iApp','Ordem']
    for c in cols_int:
        if c in df_cont.columns:
            df_cont[c] = pd.to_numeric(df_cont[c], errors='coerce').fillna(0).astype(int).astype(str)

    if tipo == "PCD":
        df_cont['param10'] = pd.to_numeric(df_cont['param10'], errors='coerce').round(1).astype(str).str.replace('.', ',')
        for c in ['param6','param12','param17','param18','param19','param11']:
            if c in df_cont.columns:
                df_cont[c] = pd.to_numeric(df_cont[c], errors='coerce').round(2).astype(str).str.replace('.', ',')
    else:
        df_cont['param10'] = pd.to_numeric(df_cont['param10'], errors='coerce').round(1).astype(str).str.replace('.', ',')
        df_cont['param11'] = pd.to_numeric(df_cont['param11'], errors='coerce').round(2).astype(str).str.replace('.', ',')
        for c in ['param6','param12']:
            df_cont[c] = pd.to_numeric(df_cont[c], errors='coerce').round(2).astype(str).str.replace('.', ',')

    df_cont['param16'] = validade_camp

    if tipo == "PCD":
        df_ex = df_cont[['param7','param11','param10','param8','param17','param6','param19']].drop_duplicates()
        df_ex.columns = ['Montante','TAN','TAEG','Prazo','Seguro','MM','MTIC']
    else:
        df_ex = df_cont[['param7','param11','param10','param8','param9','param6']].drop_duplicates()
        df_ex.columns = ['Montante','TAN','TAEG','Prazo','Seguro','MM']

    df_doss = obter_dossiers()
    df_full['NIF'] = df_full['NIF'].astype(str)
    df_doss['nif'] = df_doss['nif'].astype(str)

    df_g = df_full.merge(df_doss, left_on='NIF', right_on='nif', how='left')

    df_gestor = pd.DataFrame({
        'IDCampanha': 'NULL',
        'NumDossier': df_g['numdossier'].fillna(''),
        'Telemovel': df_g.get('telemovel', ''),
        'NIF': df_g['NIF'],
        'Tipo Oferta': tipo,
        'Montante Proposto': df_g['AMOUNT'],
        'Mensalidade Proposta': df_g['MONTHLYPAY'],
        'Tan': df_g['RATE'],
        'Taeg': df_g['APR'],
        'Ordem': 1
    })

    return df_cont, df_ex, df_gestor

# -------------------------------------------------------------------------
# PROCESSAMENTO FINAL
# -------------------------------------------------------------------------
campanhas_processadas = 0
campanhas_exportadas = 0

print(f"✓ Total de campanhas hoje: {len(campanhas_hoje)}")

for _, row in campanhas_hoje.iterrows():
    campanhas_processadas += 1
    nome = row['campanha']
    tipo = row.get('tipo oferta', '')
    necessidade = int(row['volumetria'])

    print("\n" + "="*60)
    print(f"CAMPANHA: {nome}")
    print(f"TIPO OFERTA: {tipo}")
    print(f"NECESSIDADE: {necessidade}")
    print("="*60)

    if not resultados:
        print("⚠ Sem resultados")
        continue

    df_res = pd.DataFrame(resultados)
    nifs = df_res[df_res['campanha'] == nome]['NIF'].tolist()
    df_cli = df_baseenvio[df_baseenvio['NIF'].isin(nifs)]

    if df_cli.empty:
        print("⚠ Nenhum cliente atribuído")
        continue

    if "PCD" in tipo.upper():
        df_cont, df_ex, df_g = processar_campanha(nome, df_cli, necessidade, "PCD", df_res)
    elif "RUC" in tipo.upper():
        df_cont, df_ex, df_g = processar_campanha(nome, df_cli, necessidade, "RUC", df_res)
    else:
        print("ℹ Não é PCD/RUC — ignorado")
        continue

    if df_cont is None:
        print("⚠ Nada a exportar")
        continue

    campanhas_exportadas += 1

    pasta = os.path.join(FOLDER_OUTPUT, nome.replace("iXS_", "").replace("iElegivel_", "").replace(" ", "_"))
    os.makedirs(pasta, exist_ok=True)

    df_cont.to_csv(os.path.join(pasta, f"contactos_{nome}_{hoje_file}.csv"), sep=SEPARADOR, encoding=ENCODING, index=False)
    if not df_ex.empty:
        df_ex.to_csv(os.path.join(pasta, f"exemplos_{nome}_{hoje_file}.csv"), sep=SEPARADOR, encoding=ENCODING, index=False)
    if not df_g.empty:
        df_g.to_csv(os.path.join(pasta, f"gestor_{nome}_{hoje_file}.csv"), sep=SEPARADOR, encoding=ENCODING, index=False)

    print(f"    ✓ Exportação concluída para {pasta}")

print("\n" + "="*60)
print("RESUMO DA EXPORTAÇÃO")
print("="*60)
print(f"✓ Campanhas processadas: {campanhas_processadas}")
print(f"✓ Campanhas exportadas: {campanhas_exportadas}")
print(f"✓ Não exportadas: {campanhas_processadas - campanhas_exportadas}")
print("\n✓ Processamento concluído!")



10. EXPORTAR RESULTADOS
✓ Total de campanhas hoje: 1

CAMPANHA: iXS_PCDPM_Parcerias
TIPO OFERTA: PCD
NECESSIDADE: 1090

  Processando campanha PCD: iXS_PCDPM_Parcerias


C:\Users\COTAGO\AppData\Local\Temp\ipykernel_5004\1636431686.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)
C:\Users\COTAGO\AppData\Local\Temp\ipykernel_5004\1636431686.py:107: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql(query, conn)


    ✓ Exportação concluída para C:\Users\COTAGO\Desktop\Base\Export_Campanhas\PCDPM_Parcerias

RESUMO DA EXPORTAÇÃO
✓ Campanhas processadas: 1
✓ Campanhas exportadas: 1
✓ Não exportadas: 0

✓ Processamento concluído!
